# Incorporação Imobiliária 4.0

Códigos para acessar o XML e validar os dados

## Instalações base


In [4]:
# Instalando as bibliotecas necessárias
!pip install google-colab
!pip install pandas
!apt-get install pandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
# Importando bibliotecas
from google.colab import drive
import xml.etree.ElementTree as ET
import sqlite3
import os
import pandas as pd
import sys
import subprocess
import re
import logging
import time
import locale

from pathlib import Path
from dataclasses import dataclass
from decimal import Decimal
from contextlib import ExitStack
from contextlib import contextmanager
from enum import Enum, auto
from abc import ABC, abstractmethod

from typing import Dict, List, Tuple, Optional
from typing import Dict, List, Tuple, Set, Optional
from typing import Dict, List, Optional, Any, Iterator


## Monta o GoogleDrive para uso

In [6]:
# Montando o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Notas importantes:

1. Ao executar este comando você precisará autorizar o acesso ao Google Drive clicando no link fornecido e inserindo o código de autorização.

## Acessa o arquivo XML e carrega no `root`

In [7]:
# Definindo o caminho do arquivo
arquivo_xml = '/content/drive/My Drive/Colab_cri/base_real_assinado_assinado.xml'

# Lendo o arquivo XML
try:
    tree = ET.parse(arquivo_xml)
    root = tree.getroot()
    print("Arquivo XML carregado com sucesso!")
    print(f"Elemento raiz: {root.tag}")
except FileNotFoundError:
    print("Erro: Arquivo não encontrado. Verifique se o caminho está correto.")
except ET.ParseError:
    print("Erro: O arquivo não é um XML válido.")
except Exception as e:
    print(f"Erro inesperado: {str(e)}")

Arquivo XML carregado com sucesso!
Elemento raiz: RegistroImoveis


### Notas importantes:

1. O caminho do arquivo assume que `base_real.xml` está na pasta `Colab_cri` da raiz do seu Google Drive. Se estiver em outra pasta, ajuste o caminho adequadamente.
2. O código inclui tratamento de erros para os casos mais comuns que podem ocorrer ao tentar acessar o arquivo.

## Gera um banco de dados local a partir do XML

In [8]:
# Salvar stdout original para restaurar depois
original_stdout = sys.stdout

# Configurar log
log_file = '/content/drive/My Drive/Colab_cri/log_leitura_xml.txt'
log_stdout = open(log_file, 'w')
sys.stdout = log_stdout

# Criar dicionário para DataFrames
dataframes = {}

# Processar XML para DataFrames
for table in root.findall('.//Tabela'):
    table_name = table.get('nome') if table.get('nome') else table.tag
    rows = []
    columns = []

    for row in table.findall('.//Registro'):
        row_data = {}
        for col in row:
            if col.tag not in columns:
                columns.append(col.tag)
            row_data[col.tag] = col.text
        rows.append(row_data)

    df = pd.DataFrame(rows, columns=columns)

    # Attempt to convert columns to appropriate data types
    for col in df.columns:
        # First, try to convert to numeric types
        df[col] = pd.to_numeric(df[col], errors='ignore')
        # You can also handle date columns if necessary
        # For example: df[col] = pd.to_datetime(df[col], errors='ignore')

    # Alternatively, you can use df.convert_dtypes()
    # df = df.convert_dtypes()

    dataframes[table_name] = df

# Criar e salvar no SQLite
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
with sqlite3.connect(db_path) as conn:
    for table_name, df in dataframes.items():
        # Infer SQLite data types from DataFrame dtypes
        # Build a dtype dictionary
        dtype_dict = {}
        for col in df.columns:
            if pd.api.types.is_integer_dtype(df[col]):
                dtype_dict[col] = 'INTEGER'
            elif pd.api.types.is_float_dtype(df[col]):
                dtype_dict[col] = 'REAL'
            else:
                dtype_dict[col] = 'TEXT'

        df.to_sql(table_name, conn, index=False, if_exists='replace', dtype=dtype_dict)
        print(f"Tabela {table_name} criada com {len(df)} registros")

# Restaurar stdout original antes de fechar o log
sys.stdout = original_stdout
log_stdout.close()

print("Processo concluído!")


<ipython-input-8-1e3167774795>:31: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Processo concluído!


In [9]:
# Pausar a execução por 1 minuto (60 segundos)
time.sleep(60)

### Notas sobre a criação do banco de dados:

1. O código acima lê a estrutura do seu XML e recria o banco de dados SQLite
2. Ele preserva:
   - Nomes das tabelas
   - Nomes e tipos das colunas
   - Todos os dados contidos no XML
3. O novo arquivo .db será criado na mesma pasta do arquivo XML
4. O código inclui verificações de erro e mostrará a estrutura das tabelas criadas

## Cria uma planilha de Excel da tabela "CRI" do banco de dados

In [10]:
# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Ler a tabela cri para um DataFrame
df_cri = pd.read_sql_query("SELECT * FROM cri", conn)

# Salvar como Excel
excel_path = '/content/drive/My Drive/Colab_cri/cri.xlsx'
df_cri.to_excel(excel_path, index=False)

# Fechar conexão
conn.close()

print(f"Arquivo Excel criado em: {excel_path}")
print(f"Número de registros exportados: {len(df_cri)}")

Arquivo Excel criado em: /content/drive/My Drive/Colab_cri/cri.xlsx
Número de registros exportados: 1098


### Notas sobre a criação da planilha:

1. O código acima lê a estrutura do seu banco de dados local `base_real.db` e exporta a tabela `cri` do banco de dados para o formato de Excel, `cri.xlsx` na mesma pasta `Colab_cri.py`

## Gera o memorial descritivo

### Cria a descrição por tipologia

In [11]:


# Configure logging
logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

@dataclass
class UnitData:
    """Data class to store unit information"""
    especie_unidade: str
    tipo_unidade: str
    unidade_numero: str
    area_privativa: Decimal
    area_comum: Decimal
    area_total_construida: Decimal
    fracao_ideal_solo_condominio: Decimal
    quota_terreno_condominio: Decimal
    fracao_ideal_unidade_subcondominio: Decimal
    vaga_vinculada_descoberta: str
    area_vinculada_outras: Decimal
    area_comum_descoberta: Decimal

    def __post_init__(self):
        """Convert None values to appropriate defaults"""
        self.especie_unidade = self.especie_unidade or ''
        self.tipo_unidade = self.tipo_unidade or ''
        self.unidade_numero = self.unidade_numero or ''
        self.vaga_vinculada_descoberta = self.vaga_vinculada_descoberta or '0'

        # Convert None to Decimal('0') for numeric fields
        decimal_fields = [
            'area_privativa', 'area_comum', 'area_total_construida',
            'fracao_ideal_solo_condominio', 'quota_terreno_condominio',
            'fracao_ideal_unidade_subcondominio', 'area_vinculada_outras',
            'area_comum_descoberta'
        ]

        for field in decimal_fields:
            value = getattr(self, field)
            if value is None:
                setattr(self, field, Decimal('0'))
            elif not isinstance(value, Decimal):
                setattr(self, field, Decimal(str(value)))

class UnitDescriptionGenerator:
    def __init__(self, db_path: str = './Colab_cri/base_real.db', output_dir: str = './Colab_cri/output'):
        self.db_path = Path(db_path)
        self.output_dir = Path(output_dir)
        self.conn: Optional[sqlite3.Connection] = None

        # Ensure output directory exists
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def connect_database(self) -> None:
        """Establish database connection"""
        try:
            self.conn = sqlite3.connect(str(self.db_path))
            logging.info(f"Connected to database: {self.db_path}")
        except sqlite3.Error as e:
            logging.error(f"Database connection error: {e}")
            raise

    def fetch_unit_data(self) -> Dict[Tuple[str, str], List[UnitData]]:
        """Fetch and organize unit data from database"""
        try:
            cursor = self.conn.cursor()
            query = """
            SELECT especie_unidade, tipo_unidade, unidade_numero, area_privativa,
                   area_comum, area_total_construida, fracao_ideal_solo_condominio,
                   quota_terreno_condominio, fracao_ideal_unidade_subcondominio,
                   vaga_vinculada_descoberta, area_vinculada_outras, area_comum_descoberta
            FROM cri
            WHERE especie_unidade IS NOT NULL
            ORDER BY especie_unidade, tipo_unidade;
            """
            cursor.execute(query)
            units = cursor.fetchall()

            # Log any rows with NULL especie_unidade for debugging
            cursor.execute("""
            SELECT unidade_numero, tipo_unidade
            FROM cri
            WHERE especie_unidade IS NULL;
            """)
            null_units = cursor.fetchall()
            if null_units:
                logging.warning(f"Found {len(null_units)} units with NULL especie_unidade: {null_units}")

            # Organize data by unit type
            units_dict: Dict[Tuple[str, str], List[UnitData]] = {}
            for unit in units:
                if any(x is None for x in unit[:2]):  # Check especie_unidade and tipo_unidade
                    continue

                unit_data = UnitData(*unit)
                key = (unit_data.especie_unidade, unit_data.tipo_unidade)
                if key not in units_dict:
                    units_dict[key] = []
                units_dict[key].append(unit_data)

            return units_dict

        except sqlite3.Error as e:
            logging.error(f"Error fetching unit data: {e}")
            raise

    @staticmethod
    def format_decimal(value: Decimal) -> str:
        """Format decimal values with 8 decimal places and comma separator"""
        return f"{float(value):.8f}".replace('.', ',')

    def generate_apartment_description(self, units: List[UnitData]) -> str:
        """Generate description for apartment units"""
        first_unit = units[0]
        unit_numbers = ', '.join(str(unit.unidade_numero) for unit in units)

        return f"""APARTAMENTO TIPO {first_unit.tipo_unidade}: {len(units)} unidades,
            correspondentes aos apartamentos nº {unit_numbers},
            possuindo cada unidade as seguintes áreas construídas:
            área total construída de {self.format_decimal(first_unit.area_total_construida)} metros quadrados,
            sendo a área privativa de {self.format_decimal(first_unit.area_privativa)} metros quadrados
            e a área comum de {self.format_decimal(first_unit.area_comum)} metros quadrados.
            Fração ideal nas partes comuns do subcondomínio: {self.format_decimal(first_unit.fracao_ideal_unidade_subcondominio)};
            Fração ideal de solo e partes comuns no condomínio: {self.format_decimal(first_unit.fracao_ideal_solo_condominio)};
            Quota de terreno: {self.format_decimal(first_unit.quota_terreno_condominio)} metros quadrados.
            Área comum descoberta: {self.format_decimal(first_unit.area_comum_descoberta)} metros quadrados."""

    def generate_parking_description(self, units: List[UnitData]) -> str:
        """Generate description for parking units"""
        first_unit = units[0]
        unit_numbers = ', '.join(str(unit.unidade_numero) for unit in units)

        base_text = f"""VAGA TIPO {first_unit.tipo_unidade}: {len(units)} unidades,
            correspondentes às vagas nº {unit_numbers},
            possuindo cada unidade as seguintes áreas construídas:
            área total construída de {self.format_decimal(first_unit.area_total_construida)} metros quadrados,
            sendo a área privativa de {self.format_decimal(first_unit.area_privativa)} metros quadrados
            e a área comum de {self.format_decimal(first_unit.area_comum)} metros quadrados"""

        if first_unit.area_vinculada_outras != Decimal('0'):
            base_text += f", área de depósito vinculado {unit_numbers}, respectivamente, de {self.format_decimal(first_unit.area_vinculada_outras)} metros quadrados"

        base_text += f""".
            Fração ideal nas partes comuns do subcondomínio: {self.format_decimal(first_unit.fracao_ideal_unidade_subcondominio)};
            Fração ideal de solo e partes comuns no condomínio: {self.format_decimal(first_unit.fracao_ideal_solo_condominio)};
            Quota de terreno: {self.format_decimal(first_unit.quota_terreno_condominio)} metros quadrados."""

        return base_text

    def generate_store_description(self, units: List[UnitData]) -> str:
        """Generate description for store units"""
        first_unit = units[0]
        unit_numbers = ', '.join(str(unit.unidade_numero) for unit in units)

        return f"""LOJA TIPO {first_unit.tipo_unidade}: {len(units)} unidades,
            correspondentes ao Comércio e Serviço Vicinal  nº {unit_numbers},
            possuindo cada unidade as seguintes áreas construídas:
            área total construída de {self.format_decimal(first_unit.area_total_construida)} metros quadrados,
            sendo a área privativa de {self.format_decimal(first_unit.area_privativa)} metros quadrados
            e a área comum de {self.format_decimal(first_unit.area_comum)} metros quadrados.
            Fração ideal nas partes comuns do subcondomínio: {self.format_decimal(first_unit.fracao_ideal_unidade_subcondominio)};
            Fração ideal de solo e partes comuns no condomínio: {self.format_decimal(first_unit.fracao_ideal_solo_condominio)};
            Quota de terreno: {self.format_decimal(first_unit.quota_terreno_condominio)} metros quadrados."""

    def generate_kitinete_description(self, units: List[UnitData]) -> str:
        """Generate description for kitinete units"""
        first_unit = units[0]
        unit_numbers = ', '.join(str(unit.unidade_numero) for unit in units)

        return f"""KITINETE TIPO {first_unit.tipo_unidade}: {len(units)} unidades,
            correspondentes às kitinetes nº {unit_numbers},
            possuindo cada unidade as seguintes áreas construídas:
            área total construída de {self.format_decimal(first_unit.area_total_construida)} metros quadrados,
            sendo a área privativa de {self.format_decimal(first_unit.area_privativa)} metros quadrados
            e a área comum de {self.format_decimal(first_unit.area_comum)} metros quadrados.
            Fração ideal nas partes comuns do subcondomínio: {self.format_decimal(first_unit.fracao_ideal_unidade_subcondominio)};
            Fração ideal de solo e partes comuns no condomínio: {self.format_decimal(first_unit.fracao_ideal_solo_condominio)};
            Quota de terreno: {self.format_decimal(first_unit.quota_terreno_condominio)} metros quadrados.
            Área comum descoberta: {self.format_decimal(first_unit.area_comum_descoberta)} metros quadrados."""

    def generate_descriptions(self) -> None:
        """Generate and save unit descriptions to files"""
        try:
            units_dict = self.fetch_unit_data()

            if not units_dict:
                logging.error("No valid unit data found in database")
                return

            # Process each unit type with separate file handlers
            with open(self.output_dir / 'tipos_apartamentos.txt', 'w', encoding='utf-8') as f_apartamento, \
                 open(self.output_dir / 'tipos_vagas.txt', 'w', encoding='utf-8') as f_vaga, \
                 open(self.output_dir / 'tipos_lojas.txt', 'w', encoding='utf-8') as f_loja, \
                 open(self.output_dir / 'tipos_kitinetes.txt', 'w', encoding='utf-8') as f_kitinete:

                # Process each unit type
                for (especie, tipo), units in units_dict.items():
                    try:
                        especie_lower = especie.lower() if especie else ''

                        if not especie_lower:
                            logging.warning(f"Empty especie_unidade found for tipo: {tipo}")
                            continue

                        if especie_lower == 'apartamento':
                            description = self.generate_apartment_description(units)
                            f_apartamento.write(description + '\n')
                        elif especie_lower == 'vaga':
                            description = self.generate_parking_description(units)
                            f_vaga.write(description + '\n')
                        elif especie_lower == 'loja':
                            description = self.generate_store_description(units)
                            f_loja.write(description + '\n')
                        elif especie_lower == 'kitinete':
                            description = self.generate_kitinete_description(units)
                            f_kitinete.write(description + '\n')
                        else:
                            logging.warning(f"Unknown especie_unidade: {especie}")

                    except Exception as e:
                        logging.error(f"Error processing unit type {especie} {tipo}: {e}")
                        continue

            logging.info("Unit descriptions generated successfully")

        except Exception as e:
            logging.error(f"Error generating descriptions: {e}")
            raise

    def close(self) -> None:
        """Close database connection"""
        if self.conn:
            self.conn.close()
            logging.info("Database connection closed")

def main():
    # Specify the correct path to the database and output folder
    generator = UnitDescriptionGenerator(
        db_path='/content/drive/My Drive/Colab_cri/base_real.db',
        output_dir='/content/drive/My Drive/Colab_cri/output'
    )

    try:
        generator.connect_database()
        generator.generate_descriptions()
    except Exception as e:
        logging.error(f"Application error: {e}")
        raise
    finally:
        generator.close()

if __name__ == "__main__":
    main()


### Cria a descrição por pavimento

In [12]:

import logging
import sqlite3
from pathlib import Path
from typing import Optional, List, Tuple, Dict, Set
from dataclasses import dataclass
import re

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def extract_numbers(s):
    """Extract numbers from a string for sorting."""
    nums = re.findall(r'\d+', s)
    return [int(n) for n in nums] if nums else [float('inf')]

@dataclass
class UnitSummary:
    """Class to store unit summary information"""
    total_count: int
    locations: Dict[str, List[str]]
    first_floor: Optional[str] = None
    last_floor: Optional[str] = None

class BuildingUnitLocations:
    def __init__(self, db_path: str = './content/drive/My Drive/Colab_cri/base_real.db', output_dir: str = './content/drive/My Drive/Colab_cri/output'):
        self.db_path = Path(db_path)
        self.output_dir = Path(output_dir)
        self.conn: Optional[sqlite3.Connection] = None

        # Ensure output directory exists
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def connect_database(self) -> None:
        """Establish database connection"""
        try:
            self.conn = sqlite3.connect(str(self.db_path))
            logging.info(f"Connected to database: {self.db_path}")
        except sqlite3.Error as e:
            logging.error(f"Database connection error: {e}")
            raise

    def execute_query(self, query: str, params: tuple = ()) -> List[Tuple]:
        """Execute SQL query and return results"""
        try:
            cursor = self.conn.cursor()
            cursor.execute(query, params)
            return cursor.fetchall()
        except sqlite3.Error as e:
            logging.error(f"Query execution error: {e}\nQuery: {query}")
            raise

    @staticmethod
    def extract_floor_number(floor_str: str) -> int:
        """Extract numerical floor number from string"""
        match = re.match(r"(\d+)", floor_str)
        return int(match.group(1)) if match else float('inf')

    @staticmethod
    def format_unit_numbers(units: List[str]) -> str:
        """Format unit numbers with proper separators"""
        if not units:
            return ""
        if len(units) == 1:
            return str(units[0])
        return ', '.join(map(str, units[:-1])) + f' e {units[-1]}'

    def get_parking_data(self) -> Tuple[UnitSummary, int]:
        """Fetch and process parking data"""
        total_spaces_result = self.execute_query("""
            SELECT vagas_total FROM informacoes_preliminares LIMIT 1
        """)
        total_spaces = total_spaces_result[0][0] if total_spaces_result else 0

        parking_data = self.execute_query("""
            SELECT pavimento, unidade_numero, tipo_vaga
            FROM cri
            WHERE especie_unidade = 'VAGA'
            ORDER BY pavimento, unidade_numero
        """)

        locations: Dict[str, Dict[str, List[str]]] = {}
        for floor, unit, park_type in parking_data:
            if floor not in locations:
                locations[floor] = {'simples': [], 'dupla': []}
            locations[floor][park_type].append(unit)

        return (
            UnitSummary(
                total_count=len(parking_data),
                locations=locations
            ),
            total_spaces
        )

    def get_apartment_data(self) -> UnitSummary:
        """Fetch and process apartment data"""
        apartment_data = self.execute_query("""
            SELECT pavimento, unidade_numero
            FROM cri
            WHERE especie_unidade = 'APARTAMENTO'
            ORDER BY pavimento, unidade_numero
        """)

        locations: Dict[str, List[str]] = {}
        floors: Set[str] = set()

        for floor, unit in apartment_data:
            if floor not in locations:
                locations[floor] = []
            locations[floor].append(unit)
            floors.add(floor)

        sorted_floors = sorted(floors, key=self.extract_floor_number)

        return UnitSummary(
            total_count=len(apartment_data),
            locations=locations,
            first_floor=sorted_floors[0] if sorted_floors else None,
            last_floor=sorted_floors[-1] if sorted_floors else None
        )

    def get_store_data(self) -> UnitSummary:
        """Fetch and process store data"""
        store_data = self.execute_query("""
            SELECT pavimento, unidade_numero
            FROM cri
            WHERE especie_unidade = 'LOJA'
            ORDER BY pavimento, unidade_numero
        """)

        locations: Dict[str, List[str]] = {}
        for floor, unit in store_data:
            if floor not in locations:
                locations[floor] = []
            locations[floor].append(unit)

        return UnitSummary(
            total_count=len(store_data),
            locations=locations
        )

    def get_kitinete_data(self) -> UnitSummary:
        """Fetch and process kitinete data"""
        kitinete_data = self.execute_query("""
            SELECT pavimento, unidade_numero
            FROM cri
            WHERE especie_unidade = 'KITINETE'
            ORDER BY pavimento, unidade_numero
        """)

        locations: Dict[str, List[str]] = {}
        floors: Set[str] = set()

        for floor, unit in kitinete_data:
            if floor not in locations:
                locations[floor] = []
            locations[floor].append(unit)
            floors.add(floor)

        sorted_floors = sorted(floors, key=self.extract_floor_number)

        return UnitSummary(
            total_count=len(kitinete_data),
            locations=locations,
            first_floor=sorted_floors[0] if sorted_floors else None,
            last_floor=sorted_floors[-1] if sorted_floors else None
        )

    def generate_parking_text(self, summary: UnitSummary, total_spaces: int) -> str:
        """Generate formatted text for parking spaces"""
        simple_count = sum(len(floor_data['simples']) for floor_data in summary.locations.values())
        double_count = sum(len(floor_data['dupla']) for floor_data in summary.locations.values())

        text = (f"As vagas localizam-se nos {', '.join(summary.locations.keys())}, "
                f"num total de {summary.total_count} unidades autônomas, "
                f"comportando {total_spaces} veículos sendo {simple_count} "
                f"vagas simples e {double_count} vagas duplas.\n")

        for floor in sorted(summary.locations.keys()):
            floor_data = summary.locations[floor]
            simple_units = sorted(floor_data['simples'], key=extract_numbers)
            double_units = sorted(floor_data['dupla'], key=extract_numbers)
            simple = self.format_unit_numbers(simple_units)
            double = self.format_unit_numbers(double_units)

            text += f"\nNo {floor} serão {len(floor_data['simples']) + len(floor_data['dupla'])} "
            text += f"vagas autônomas de nº {simple} (vagas simples)"
            if double_units:
                text += f" e {double} (vagas duplas)"
            text += "."

        return text

    def generate_apartment_text(self, summary: UnitSummary) -> str:
        """Generate formatted text for apartments"""
        text = (f"Os apartamentos estão localizados do {summary.first_floor} ao {summary.last_floor}, "
                f"num total de {summary.total_count} unidades autônomas, sendo:\n")

        for floor in sorted(summary.locations.keys(), key=self.extract_floor_number):
            units = self.format_unit_numbers(sorted(summary.locations[floor], key=extract_numbers))
            text += f"{floor}: apartamentos nº {units};\n"

        return text

    def generate_store_text(self, summary: UnitSummary) -> str:
        """Generate formatted text for stores"""
        floors = list(summary.locations.keys())

        if not floors:
            return "Não há lojas disponíveis."

        first_floor_text = floors[0]
        units_first_floor = sorted(summary.locations[first_floor_text], key=extract_numbers)
        units_on_the_first_floor = ", ".join(units_first_floor)

        if len(floors) > 1:
            second_floor_text = floors[1]
            units_second_floor = sorted(summary.locations[second_floor_text], key=extract_numbers)
            units_on_the_second_floor = ", ".join(units_second_floor)
            text = (f"As lojas, num total de {summary.total_count} "
                    f"unidades autônomas, são as Lojas nº {units_on_the_first_floor}, {units_on_the_second_floor}. "
                    f"Destas, as Lojas nº {units_on_the_first_floor} estão no {first_floor_text} e as "
                    f"Lojas nº {units_on_the_second_floor} estão no {second_floor_text}.")
        else:
            text = (f"As lojas, num total de {summary.total_count} "
                    f"unidades autônomas, são as Lojas nº {units_on_the_first_floor} "
                    f"no {first_floor_text}.")

        return text

    def generate_kitinete_text(self, summary: UnitSummary) -> str:
        """Generate formatted text for kitinetes"""
        text = (f"As kitinetes estão localizadas do {summary.first_floor} ao {summary.last_floor}, "
                f"num total de {summary.total_count} unidades autônomas, sendo:\n")

        for floor in sorted(summary.locations.keys(), key=self.extract_floor_number):
            units = self.format_unit_numbers(sorted(summary.locations[floor], key=extract_numbers))
            text += f"{floor}: kitinetes nº {units};\n"

        return text

    def generate_all_texts(self) -> None:
        """Generate and save all unit location texts"""
        try:
            # Get data
            parking_summary, total_spaces = self.get_parking_data()
            apartment_summary = self.get_apartment_data()
            store_summary = self.get_store_data()
            kitinete_summary = self.get_kitinete_data()

            # Generate texts
            parking_text = self.generate_parking_text(parking_summary, total_spaces)
            apartment_text = self.generate_apartment_text(apartment_summary)
            store_text = self.generate_store_text(store_summary)
            kitinete_text = self.generate_kitinete_text(kitinete_summary)

            # Save texts
            text_files = {
                'localizacao_vagas.txt': parking_text,
                'localizacao_apartamentos.txt': apartment_text,
                'localizacao_lojas.txt': store_text,
                'localizacao_kitinetes.txt': kitinete_text
            }

            for filename, content in text_files.items():
                filepath = self.output_dir / filename
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(content)
                logging.info(f"Generated {filename}")

            # Print texts (optional)
            print(parking_text)
            print(apartment_text)
            print(store_text)
            print(kitinete_text)

        except Exception as e:
            logging.error(f"Error generating texts: {e}")
            raise

    def close(self) -> None:
        """Close database connection"""
        if self.conn:
            self.conn.close()
            logging.info("Database connection closed")


def main():
    # Specify the correct path to the database and output folder
    generator = BuildingUnitLocations(
        db_path='/content/drive/My Drive/Colab_cri/base_real.db',
        output_dir='/content/drive/My Drive/Colab_cri/output'
    )

    try:
        generator.connect_database()
        generator.generate_all_texts()
    except Exception as e:
        logging.error(f"Application error: {e}")
        raise
    finally:
        generator.close()

if __name__ == "__main__":
    main()


As vagas localizam-se nos subsolo 1, subsolo 2, subsolo 3, num total de 208 unidades autônomas, comportando 257 veículos sendo 159 vagas simples e 49 vagas duplas.

No subsolo 1 serão 64 vagas autônomas de nº 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66 e 67 (vagas simples) e 18/17, 20/19 e 22/21 (vagas duplas).
No subsolo 2 serão 72 vagas autônomas de nº 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 95, 96, 97, 98, 99, 100, 101, 122, 123, 124, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161 e 162 (vagas simples) e 80/79, 82/81, 84/83, 86/85, 88/87, 90/89, 92/91, 94/93, 102/103, 104/105, 106/107, 108/109, 110/111, 113/112, 114/115, 116/117, 118/119, 120/121, 125/126, 127/128, 129/130, 146/147 e 148/149 (vagas duplas).
No 

### Cria o memorial descritivo

In [39]:

import logging
import sqlite3
import sys
from pathlib import Path
from typing import Dict, Any, Iterator, List, Tuple, Optional
from decimal import Decimal, InvalidOperation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from contextlib import contextmanager
from enum import Enum, auto
import os
import re

# Configure logging with more detailed format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    handlers=[
        logging.FileHandler('memorial_generator.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger(__name__)

class MemorialError(Exception):
    """Base exception for memorial generator errors"""
    pass

class DatabaseError(MemorialError):
    """Database-related errors"""
    pass

class FileError(MemorialError):
    """File handling errors"""
    pass

class UnitType(Enum):
    """Enumeration for unit types"""
    APARTMENT = auto()
    PARKING = auto()
    STORE = auto()
    UNKNOWN = auto()

    @classmethod
    def from_str(cls, value: str) -> 'UnitType':
        """Convert string to UnitType"""
        mapping = {
            'APARTAMENTO': cls.APARTMENT,
            'KITINETE': cls.APARTMENT,  # Map KITINETE to APARTMENT
            'VAGA': cls.PARKING,
            'LOJA': cls.STORE
        }
        return mapping.get(value.upper(), cls.UNKNOWN)

@dataclass
class UnitData:
    """Data class to store unit information with enhanced type safety"""
    especie_unidade: str
    unidade_numero: str
    subcondominio: str
    area_total_construida: Decimal
    area_privativa: Decimal
    area_comum: Decimal
    fracao_ideal_solo_condominio: Decimal
    fracao_ideal_unidade_subcondominio: Decimal
    quota_terreno_condominio: Decimal
    area_comum_descoberta: Decimal
    pavimento: str
    confrontacao_frente: str
    confrontacao_direita: str
    confrontacao_esquerda: str
    confrontacao_fundos: str
    tipo_vaga: Optional[str] = None
    area_vinculada_outras: Decimal = Decimal('0.0')

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'UnitData':
        """Create UnitData instance from dictionary with validation"""
        try:
            return cls(
                especie_unidade=str(data.get('especie_unidade', '')).upper(),
                unidade_numero=str(data.get('unidade_numero', '')),
                subcondominio=str(data.get('subcondominio', '')).upper(),
                area_total_construida=Decimal(str(data.get('area_total_construida', 0))).quantize(Decimal('0.00000000')),
                area_privativa=Decimal(str(data.get('area_privativa', 0))).quantize(Decimal('0.00000000')),
                area_comum=Decimal(str(data.get('area_comum', 0))).quantize(Decimal('0.00000000')),
                fracao_ideal_solo_condominio=Decimal(str(data.get('fracao_ideal_solo_condominio', 0))).quantize(Decimal('0.00000000')),
                fracao_ideal_unidade_subcondominio=Decimal(str(data.get('fracao_ideal_unidade_subcondominio', 0))).quantize(Decimal('0.00000000')),
                quota_terreno_condominio=Decimal(str(data.get('quota_terreno_condominio', 0))).quantize(Decimal('0.00000000')),
                area_comum_descoberta=Decimal(str(data.get('area_comum_descoberta', 0))).quantize(Decimal('0.00000000')),
                pavimento=str(data.get('pavimento', '')),
                confrontacao_frente=str(data.get('confrontacao_frente', '')),
                confrontacao_direita=str(data.get('confrontacao_direita', '')),
                confrontacao_esquerda=str(data.get('confrontacao_esquerda', '')),
                confrontacao_fundos=str(data.get('confrontacao_fundos', '')),
                tipo_vaga=data.get('tipo_vaga'),
                area_vinculada_outras=Decimal(str(data.get('area_vinculada_outras', 0))).quantize(Decimal('0.00000000'))
            )
        except (ValueError, TypeError, InvalidOperation) as e:
            raise ValueError(f"Error creating UnitData from dictionary: {e}")

def format_decimal(value: Decimal, decimal_places: int = 8) -> str:
    """Format decimal values with specified decimal places and comma separator"""
    formatted = f"{value:.{decimal_places}f}".replace('.', ',')
    return formatted

class UnitDescriptionGenerator(ABC):
    """Abstract base class for unit description generators"""
    @abstractmethod
    def generate(self, unit: UnitData) -> str:
        """Generate description for a unit"""
        pass

class ApartmentDescriptionGenerator(UnitDescriptionGenerator):
    def generate(self, unit: UnitData) -> str:
        unit_type_name = unit.especie_unidade.capitalize()
        return f"""{unit_type_name} {unit.unidade_numero}:
Subcondomínio: {unit.subcondominio}.
Áreas construídas: área total construída de {format_decimal(unit.area_total_construida)} metros quadrados,
sendo a área privativa de {format_decimal(unit.area_privativa)} metros quadrados
e a área comum de {format_decimal(unit.area_comum)} metros quadrados.
Fração ideal nas partes comuns do subcondomínio: {format_decimal(unit.fracao_ideal_unidade_subcondominio)};
Fração ideal de solo e partes comuns no condomínio: {format_decimal(unit.fracao_ideal_solo_condominio)};
Quota de terreno: {format_decimal(unit.quota_terreno_condominio)} metros quadrados.
Area comum descoberta: {format_decimal(unit.area_comum_descoberta)} metros quadrados.
Localização: {unit.pavimento}, sendo que para quem entra na unidade,
confronta pela frente com {unit.confrontacao_frente},
pelo lado direito com {unit.confrontacao_direita},
pelo lado esquerdo com {unit.confrontacao_esquerda}
e pelo fundo com {unit.confrontacao_fundos}."""

class ParkingDescriptionGenerator(UnitDescriptionGenerator):
    def generate(self, unit: UnitData) -> str:
        tipo_vaga_str = "simples para (1)" if unit.tipo_vaga and unit.tipo_vaga.lower() == 'simples' else "dupla para (2)"

        description = [
            f"VAGA {unit.unidade_numero}:",
            f"Subcondomínio: {unit.subcondominio}.",
            f"Capacidade e uso: {tipo_vaga_str} veículo(s) de passeio, de pequeno e médio porte.",
            "Áreas construídas:",
            f"área total construída de {format_decimal(unit.area_total_construida)} metros quadrados,",
            f"sendo a área privativa de {format_decimal(unit.area_privativa)} metros quadrados"
        ]

        if unit.area_vinculada_outras > 0:
            description.append(f", área de depósito nº {unit.unidade_numero} de {format_decimal(unit.area_vinculada_outras)} metros quadrados")

        description.extend([
            f", e a área comum de {format_decimal(unit.area_comum)} metros quadrados.",
            f"Fração ideal nas partes comuns do subcondomínio: {format_decimal(unit.fracao_ideal_unidade_subcondominio)};",
            f"Fração ideal de solo e partes comuns no condomínio: {format_decimal(unit.fracao_ideal_solo_condominio)};",
            f"Quota de terreno: {format_decimal(unit.quota_terreno_condominio)} metros quadrados.",
            f"Localização: {unit.pavimento}, sendo que para quem entra na unidade,",
            f"confronta pela frente com {unit.confrontacao_frente},",
            f"pelo lado direito com {unit.confrontacao_direita},",
            f"pelo lado esquerdo com {unit.confrontacao_esquerda},",
            f"e pelo fundo com {unit.confrontacao_fundos}."
        ])

        return ' '.join(description)

class StoreDescriptionGenerator(UnitDescriptionGenerator):
    def generate(self, unit: UnitData) -> str:
        return f"""LOJA {unit.unidade_numero}:
Subcondomínio: {unit.subcondominio}.
Áreas construídas: área total construída de {format_decimal(unit.area_total_construida)} metros quadrados,
sendo a área privativa de {format_decimal(unit.area_privativa)} metros quadrados
e a área comum de {format_decimal(unit.area_comum)} metros quadrados.
Fração ideal nas partes comuns do subcondomínio: {format_decimal(unit.fracao_ideal_unidade_subcondominio)};
Fração ideal de solo e partes comuns no condomínio: {format_decimal(unit.fracao_ideal_solo_condominio)};
Quota de terreno: {format_decimal(unit.quota_terreno_condominio)} metros quadrados.
Localização: {unit.pavimento}, sendo que para quem entra na unidade,
confronta pela frente com {unit.confrontacao_frente},
pelo lado direito com {unit.confrontacao_direita},
pelo lado esquerdo com {unit.confrontacao_esquerda}
e pelo fundo com {unit.confrontacao_fundos}."""

def extract_numbers(s):
    """Extract numbers from a string for sorting."""
    nums = re.findall(r'\d+', s)
    return [int(n) for n in nums] if nums else [float('inf')]

@contextmanager
def database_connection(db_path: Path) -> Iterator[sqlite3.Connection]:
    """Context manager for database connections"""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        conn.row_factory = sqlite3.Row
        yield conn
    except sqlite3.Error as e:
        raise DatabaseError(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

class MemorialGenerator:
    def __init__(self, db_path: Path, output_dir: Path):
        self.db_path = db_path
        self.output_dir = output_dir
        self.description_generators = {
            UnitType.APARTMENT: ApartmentDescriptionGenerator(),
            UnitType.PARKING: ParkingDescriptionGenerator(),
            UnitType.STORE: StoreDescriptionGenerator()
        }

        # Ensure output directory exists
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def generate_memorial(self) -> None:
        """Generate complete memorial description with improved error handling"""
        try:
            with database_connection(self.db_path) as conn:
                # Add the LOJA check here
                self.check_loja_entries(conn)

                units_data = self._fetch_unit_data(conn)
                if not units_data:
                    raise DatabaseError("No unit data found in database")

                units_with_descriptions = self._generate_descriptions(units_data)
                markdown_content, residencial_content, galeria_content, estacionamento_content = self._generate_markdown_content(units_with_descriptions, units_data[0])

                # Write the main memorial markdown file
                memorial_path = self.output_dir / 'memorial.md'
                memorial_path.write_text(markdown_content, encoding='utf-8')
                logger.info(f"Memorial description saved to: {memorial_path}")

                # Write the subcondominium content files
                residencial_path = self.output_dir / 'residencial.txt'
                residencial_path.write_text(residencial_content.strip(), encoding='utf-8')
                logger.info(f"Residencial content saved to: {residencial_path}")

                galeria_path = self.output_dir / 'galeria.txt'
                galeria_path.write_text(galeria_content.strip(), encoding='utf-8')
                logger.info(f"Galeria content saved to: {galeria_path}")

                estacionamento_path = self.output_dir / 'estacionamento.txt'
                estacionamento_path.write_text(estacionamento_content.strip(), encoding='utf-8')
                logger.info(f"Estacionamento content saved to: {estacionamento_path}")

            # No need to write acesso_edificio.txt here, it's handled in _generate_markdown_content

        except Exception as e:
            raise MemorialError(f"Error generating memorial: {e}")

    def check_loja_entries(self, conn: sqlite3.Connection) -> None:
        """Check LOJA entries in database"""
        try:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT unidade_numero, especie_unidade, subcondominio, pavimento
                FROM cri
                WHERE especie_unidade = 'LOJA'
                ORDER BY unidade_numero
            """)
            lojas = cursor.fetchall()

            logger.info(f"Found {len(lojas)} LOJA entries in database:")
            for loja in lojas:
                logger.info(f"LOJA {loja['unidade_numero']} - Subcondominio: {loja['subcondominio']} - Pavimento: {loja['pavimento']}")

        except sqlite3.Error as e:
            logger.error(f"Error checking LOJA entries: {e}")

    def _fetch_unit_data(self, conn: sqlite3.Connection) -> List[Dict[str, Any]]:
        """Fetch unit data with improved error handling and duplicate checking"""
        try:
            cursor = conn.cursor()

            # First, let's check for duplicates in the database
            cursor.execute("""
                SELECT unidade_numero, COUNT(*) as count
                FROM cri
                GROUP BY unidade_numero
                HAVING count > 1
            """)

            duplicates = cursor.fetchall()
            if duplicates:
                for unit in duplicates:
                    logger.warning(f"Found {unit['count']} duplicates for unit {unit['unidade_numero']}")

            # Fetch all units
            cursor.execute("SELECT * FROM cri")
            units = [dict(row) for row in cursor.fetchall()]

            # Log each unit being fetched
            unit_numbers = [unit['unidade_numero'] for unit in units]
            logger.info(f"Fetched {len(units)} units: {', '.join(unit_numbers)}")

            # Sort units by number using extract_numbers
            sorted_units = sorted(units, key=lambda x: extract_numbers(x.get('unidade_numero', '0')))
            return sorted_units

        except sqlite3.Error as e:
            raise DatabaseError(f"Error fetching unit data: {e}")

    def _generate_descriptions(self, units_data: List[Dict[str, Any]]) -> List[Tuple[UnitData, str]]:
        """Generate descriptions for all units with added logging"""
        units_with_descriptions = []
        unit_counts = {'APARTAMENTO': 0, 'KITINETE': 0, 'VAGA': 0, 'LOJA': 0}

        for unit_dict in units_data:
            try:
                # Log raw unit data
                logger.info(f"Processing unit: {unit_dict.get('especie_unidade')} - {unit_dict.get('unidade_numero')}")

                unit_data = UnitData.from_dict(unit_dict)
                unit_type = UnitType.from_str(unit_data.especie_unidade)

                # Count each type
                unit_counts[unit_data.especie_unidade] = unit_counts.get(unit_data.especie_unidade, 0) + 1

                generator = self.description_generators.get(unit_type)
                if generator is None:
                    logger.warning(f"Unknown unit type: {unit_data.especie_unidade}")
                    continue

                description = generator.generate(unit_data)
                units_with_descriptions.append((unit_data, description))
                logger.info(f"Generated description for {unit_data.especie_unidade} {unit_data.unidade_numero}")

            except Exception as e:
                logger.error(f"Error generating description for unit {unit_dict.get('unidade_numero')}: {e}")
                continue

        logger.info(f"Unit counts: {unit_counts}")
        logger.info(f"Total descriptions generated: {len(units_with_descriptions)}")
        return units_with_descriptions

    def _generate_markdown_content(self, units_with_descriptions: List[Tuple[UnitData, str]], first_unit: Dict[str, Any]) -> Tuple[str, str, str, str]:
        """Generate complete markdown content and separate content for each subcondominium"""
        # Group descriptions by subcondominium
        grouped_descriptions = {
            'SUBCONDOMINIO RESIDENCIAL': [],
            'SUBCONDOMINIO GALERIA': [],
            'SUBCONDOMINIO ESTACIONAMENTO': []
        }

        # Log all descriptions before grouping
        logger.info(f"Total descriptions to process: {len(units_with_descriptions)}")

        for unit_data, desc in units_with_descriptions:
            if unit_data.especie_unidade in ['APARTAMENTO', 'KITINETE']:
                group = 'SUBCONDOMINIO RESIDENCIAL'
            elif unit_data.especie_unidade == 'LOJA':
                group = 'SUBCONDOMINIO GALERIA'
            elif unit_data.especie_unidade == 'VAGA':
                group = 'SUBCONDOMINIO ESTACIONAMENTO'
            else:
                logger.warning(f"Unknown especie_unidade: {unit_data.especie_unidade}")
                continue

            grouped_descriptions[group].append((unit_data, desc))

        # Sort the descriptions to ensure consistent ordering
        for group in grouped_descriptions:
            grouped_descriptions[group].sort(key=lambda x: extract_numbers(x[0].unidade_numero))

        # Generate content for each group
        residencial_content = os.linesep + (os.linesep + os.linesep).join(desc for _, desc in grouped_descriptions['SUBCONDOMINIO RESIDENCIAL'])
        galeria_content = os.linesep + (os.linesep + os.linesep).join(desc for _, desc in grouped_descriptions['SUBCONDOMINIO GALERIA'])
        estacionamento_content = os.linesep + (os.linesep + os.linesep).join(desc for _, desc in grouped_descriptions['SUBCONDOMINIO ESTACIONAMENTO'])

        # Read external files
        localizacao_residencial = self._read_external_file('/content/drive/My Drive/Colab_cri/output/localizacao_apartamentos.txt')
        localizacao_estacionamento = self._read_external_file('/content/drive/My Drive/Colab_cri/output/localizacao_vagas.txt')
        localizacao_galeria = self._read_external_file('/content/drive/My Drive/Colab_cri/output/localizacao_lojas.txt')
        tipos_residencial = self._read_external_file('/content/drive/My Drive/Colab_cri/output/tipos_apartamentos.txt')
        tipos_estacionamento = self._read_external_file('/content/drive/My Drive/Colab_cri/output/tipos_vagas.txt')
        tipos_galeria = self._read_external_file('/content/drive/My Drive/Colab_cri/output/tipos_lojas.txt')

        # Prepare the 'acesso_edificio' content
        acesso_edificio_content = first_unit.get('acesso_edificio', 'N/A').strip()

        # Write the 'acesso_edificio.txt' file
        acesso_edificio_path = self.output_dir / 'acesso_edificio.txt'
        acesso_edificio_path.write_text(acesso_edificio_content, encoding='utf-8')
        logger.info(f"Acesso ao edificio content saved to: {acesso_edificio_path}")

        markdown_content = f"""

# DADOS GERAIS
## Incorporador
{first_unit.get('incorporador', 'N/A')}

## Responsável técnico pela construção
{first_unit.get('responsavel_tecnico_construcao', 'N/A')}

## Responsável técnico pelo cálculo áreas NBR 12.721
{first_unit.get('responsavel_tecnico_nbr', 'N/A')}

## Matrícula
{first_unit.get('matricula', 'N/A')}

## Edifício
{first_unit.get('edificio', 'N/A')}

## Acesso ao edificio
{acesso_edificio_content}

# PARTES DE PROPRIEDADE EXCLUSIVA

São partes de propriedade exclusiva as seguintes unidades autônomas, agrupadas por subcondomínio.

## SUBCONDOMINIO RESIDENCIAL
{residencial_content}

## SUBCONDOMINIO GALERIA
{galeria_content}

## SUBCONDOMINIO ESTACIONAMENTO
{estacionamento_content}

# PARTES COMUNS
{first_unit.get('partes_comuns_base', 'N/A')}

## CONDOMINIO GERAL
{first_unit.get('partes_comuns_geral', 'N/A')}

## SUBCONDOMINIO RESIDENCIAL
{first_unit.get('partes_comuns_residencial', 'N/A')}

## SUBCONDOMINIO ESTACIONAMENTO
{first_unit.get('partes_comuns_estacionamento', 'N/A')}

## SUBCONDOMINIO RESIDENCIAL E ESTACIONAMENTO
{first_unit.get('partes_comuns_residencial_estacionamento', 'N/A')}

## SUBCONDOMINIO GALERIA
{first_unit.get('partes_comuns_galeria', 'N/A')}

# LOCALIZAÇÃO DAS UNIDADES AUTÔNMAS
## SUBCONDOMINIO RESIDENCIAL
{localizacao_residencial}

## SUBCONDOMINIO ESTACIONAMENTO
{localizacao_estacionamento}

## SUBCONDOMINIO GALERIA
{localizacao_galeria}

# TIPOLOGIA DAS UNIDADES AUTÔNMAS
## SUBCONDOMINIO RESIDENCIAL
{tipos_residencial}

## SUBCONDOMINIO ESTACIONAMENTO
{tipos_estacionamento}

## SUBCONDOMINIO GALERIA
{tipos_galeria}

******** Fim do documento ************
"""

        return markdown_content, residencial_content, galeria_content, estacionamento_content

    @staticmethod
    def _read_external_file(file_path: str) -> str:
        try:
            absolute_path = Path(file_path).absolute()
            logger.info(f"Reading file from: {absolute_path}")
            content = Path(file_path).read_text(encoding='utf-8').strip()
            logger.info(f"Content read from {file_path}:\n{content[:200]}...")
            return "\n".join(line.lstrip() for line in content.splitlines())
        except FileNotFoundError:
            logger.warning(f"File not found: {file_path}")
            return "File not found."
        except Exception as e:
            logger.error(f"Error reading file {file_path}: {e}")
            return f"Error reading file: {e}"

def main() -> None:
    """Main function with improved error handling and logging"""
    try:
        # Configuration
        db_path = Path('/content/drive/My Drive/Colab_cri/base_real.db')
        output_dir = Path('/content/drive/My Drive/Colab_cri/output')

        # Generate memorial
        generator = MemorialGenerator(db_path, output_dir)
        generator.generate_memorial()

        logger.info("Memorial generation completed successfully")

    except MemorialError as e:
        logger.error(f"Memorial generation failed: {e}")
        sys.exit(1)
    except Exception as e:
        logger.critical(f"Unexpected error: {e}", exc_info=True)
        sys.exit(1)

if __name__ == "__main__":
    main()





### Converte o memorial em markdown para arquivo Word (.docx)

In [14]:


def convert_md_to_docx(input_file, output_file, reference_docx='/content/drive/My Drive/Colab_cri/reference.docx'):
    try:
        # Using reference-doc parameter to apply the template
        subprocess.run([
            'pandoc',
            input_file,
            '-o', output_file,
            '--reference-doc', reference_docx,
            '--standalone'  # Ensures complete document structure
        ], check=True)
        print(f"File {output_file} created successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error converting {input_file} to {output_file}: {e}")

def convert_docx_to_pdf(input_file, output_file):
    try:
        subprocess.run(['pandoc', input_file, '-o', output_file], check=True)
        print(f"File {output_file} created successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error converting {input_file} to {output_file}: {e}")


# Path to your markdown file
md_file = '/content/drive/My Drive/Colab_cri/output/memorial.md'
docx_file = '/content/drive/My Drive/Colab_cri/output/memorial.docx'

# Convert markdown to docx using the reference document
convert_md_to_docx(md_file, docx_file)

File /content/drive/My Drive/Colab_cri/output/memorial.docx created successfully.


### Notas sobre a criação do memorial descritivo:

1.
2.

## Gera os quadros de área da NBR

### Gera o quadro NBR 00 (Informações Preliminares)

In [15]:

# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Buscar as informações da tabela informacoes_preliminares com base nas variáveis fornecidas
query = """
SELECT nome_incorporador, cnpj_incorporador, endereco_incorporador,
       nome_responsavel_tecnico, registro_crea, art, endereco_responsavel_tecnico,
       nome_edificio, local_construcao, cidade_uf, designacao_projeto_padrao,
       quantidade_unidades_autonomas, padrao_acabamento, numero_pavimentos, vagas_total,
       vagas_unidade_autonoma, vagas_acessorio_unidade_autonoma, vagas_uso_comum, area_lote,
       data_aprovacao_projeto, numero_alvara_projeto, nota
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas
incorporador = result['nome_incorporador']
cnpj = result['cnpj_incorporador']
endereco_incorporador = result['endereco_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
art = result['art']
endereco_responsavel_tecnico = result['endereco_responsavel_tecnico']
nome_edificio = result['nome_edificio']
local_construcao = result['local_construcao']
cidade_uf = result['cidade_uf']
designacao_projeto_padrao = result['designacao_projeto_padrao']
quantidade_unidades_autonomas = result['quantidade_unidades_autonomas']
padrao_acabamento = result['padrao_acabamento']
numero_pavimentos = result['numero_pavimentos']
vagas_total = result['vagas_total']
vagas_unidade_autonoma = result['vagas_unidade_autonoma']
vagas_acessorio_unidade_autonoma = result['vagas_acessorio_unidade_autonoma']
vagas_uso_comum = result['vagas_uso_comum']
area_lote = result['area_lote']
data_aprovacao_projeto = result['data_aprovacao_projeto']
numero_alvara_projeto = result['numero_alvara_projeto']
nota = result['nota']

# Fechar a conexão com o banco de dados
conn.close()

# HTML para o novo quadro com a estrutura ajustada de acordo com a imagem de referência
html_quadro = f"""
<html>
<table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
            INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
            (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
        </td>
    </tr>
    <tr>
        <td colspan="4" style="text-align: center; border: 1px solid black;">
            NBR 12.721 - INFORMAÇÕES PRELIMINARES
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Local do Imóvel: {local_construcao}, Curitiba, Paraná</td>
        <td style="border: 1px solid black;">Folha 1</td>
        <td style="border: 1px solid black;">Total de Folhas 10</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Nome do edifício: {nome_edificio}</td>
    </tr>
    <tr>
        <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
        <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">{incorporador}</td>
        <td style="border: 1px solid black;">Data: 28/10/2024</td>
        <td style="border: 1px solid black;">{responsavel_tecnico} ({registro_crea})</td>
        <td style="border: 1px solid black;"></td>
    </tr>
</table>
<body>
    <h1 style="text-align: center;">NBR 12.721 - INFORMAÇÕES PRELIMINARES</h1>
    <tr>
        <td style="text-align: left;">Folha 1</td>
        <td style="text-align: left;">Total de Folhas 10</td>
    </tr>
    <table>
        <tr class="section-header">
            <td colspan="2">1. INCORPORADOR</td>
        </tr>
        <tr>
            <td>1.1 Nome:</td>
            <td>{incorporador}</td>
        </tr>
        <tr>
            <td>1.2 CNPJ / CPF:</td>
            <td>{cnpj}</td>
        </tr>
        <tr>
            <td>1.3 Endereço:</td>
            <td>{endereco_incorporador}</td>
        </tr>

        <tr class="section-header">
            <td colspan="4">2. RESPONSABILIDADE TÉCNICA PELAS INFORMAÇÕES E CÁLCULOS</td>
        </tr>
        <tr>
            <td>2.1 Profissional Responsável Técnico:</td>
            <td>{responsavel_tecnico}</td>
        </tr>
        <tr>
            <td>2.2 Número de registro profissional no CREA:</td>
            <td>{registro_crea}</td>
        </tr>
        <tr>
            <td>2.3 Anotação de Responsabilidade Técnica (ART):</td>
            <td>{art}</td>
        </tr>
        <tr>
            <td>2.4 Endereço:</td>
            <td>{endereco_responsavel_tecnico}</td>
        </tr>

        <tr class="section-header">
            <td colspan="4">3. DADOS DO PROJETO / IMÓVEL</td>
        </tr>
        <tr>
            <td>3.1 Nome do Edifício:</td>
            <td>{nome_edificio}</td>
        </tr>
        <tr>
            <td>3.2 Local da Construção:</td>
            <td>{local_construcao}</td>
        </tr>
        <tr>
            <td>3.3 Cidade / UF:</td>
            <td>{cidade_uf}</td>
        </tr>
        <tr>
            <td>3.4 Designação Projeto-padrão da NBR 12.721:</td>
            <td>{designacao_projeto_padrao}</td>
        </tr>
        <tr>
            <td>3.5 Quantidade de Unidades Autônomas:</td>
            <td>{quantidade_unidades_autonomas}</td>
        </tr>
        <tr>
            <td>3.6 Padrão de Acabamento:</td>
            <td>{padrao_acabamento}</td>
        </tr>
        <tr>
            <td>3.7 Número de Pavimentos:</td>
            <td>{numero_pavimentos}</td>
        </tr>
        <tr>
            <td>3.8 Vagas Total:</td>
            <td>{vagas_total}</td>
        </tr>
        <tr>
            <td>3.8.1 Vagas por Unidade Autônoma:</td>
            <td>{vagas_unidade_autonoma}</td>
        </tr>
        <tr>
            <td>3.8.2 Vagas Acessório de Unidade Autônoma:</td>
            <td>{vagas_acessorio_unidade_autonoma} (vagas presas)</td>
        </tr>
        <tr>
            <td>3.8.3 Vagas de Uso Comum:</td>
            <td>{vagas_uso_comum}</td>
        </tr>
        <tr>
            <td>3.9 Área do Lote / Terreno:</td>
            <td>{area_lote} m²</td>
        </tr>
        <tr>
            <td>3.10 Data de aprovação do projeto arquitetônico:</td>
            <td>{data_aprovacao_projeto}</td>
        </tr>
        <tr>
            <td>3.11 Número do Alvará:</td>
            <td>{numero_alvara_projeto}</td>
        </tr>

        <tr class="section-header">
            <td colspan="2">4. INFORMACOES PLANILHAS / QUADROS</td>
        </tr>
            <td colspan="2">
            Esta é a primeira folha de um total de 10 folhas,
            todas numeradas seguidamente e assinadas conjuntamente
            pelo profissional responsável técnico, incorporador /
            proprietário, para arquivamento e registro junto ao
            competente Registro de Imóveis, em atendimento ao disposto
            na Lei 4.591, de 16 de dezembro de 1.964.
            </td>
        </tr>
        <tr>
            <td>Nota:</td>
            <td>{nota}</td>
        </tr>
    </table>
</body>
</html>
"""

# Salvar o conteúdo HTML em um arquivo
with open("/content/drive/My Drive/Colab_cri/output/nbr_00_informacoes_preliminares.html", "w") as file:
    file.write(html_quadro)

print("Arquivo HTML gerado com sucesso.")


Arquivo HTML gerado com sucesso.


### Gera o quadro NBR 01

In [16]:
# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela quadro_area_01
query = "SELECT * FROM quadro_area_01"
df_quadro_area_01 = pd.read_sql_query(query, conn)

# Buscar as informações da tabela informacoes_preliminares
query = """
SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas
incorporador = result['nome_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
local_construcao = result['local_construcao']
nome_edificio = result['nome_edificio']


# Fechar a conexão com o banco de dados
conn.close()

# Criar as colunas hierárquicas de acordo com a estrutura fornecida
columns = pd.MultiIndex.from_tuples([
    # pavimento
    ("", "", "", "Pavimento", "1"),

    # ÁREA DE DIVISÃO NÃO PROPORCIONAL: ÁREA PRIVATIVA
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO", "", "2"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "REAL", "3"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "EQUIVALENTE", "4"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "TOTAIS", "REAL (2+3)", "5"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (2+4)", "6"),

    # ÁREA DE DIVISÃO NÃO PROPORCIONAL: ÁREA DE USO COMUM
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO", "", "7"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "REAL", "8"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "EQUIVALENTE", "9"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "REAL (7+8)", "10"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (7+9)", "11"),

    # ÁREA DE DIVISÃO PROPORCIONAL: ÁREA DE USO COMUM
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO", "", "12"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "REAL", "13"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "EQUIVALENTE", "14"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "REAL (12+13)", "15"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (12+14)", "16"),

    # ÁREA DO PAVIMENTO
    ("ÁREA DO PAVIMENTO", "", "", "REAL 5+10+15)", "17"),
    ("ÁREA DO PAVIMENTO", "", "", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (6+11+16)", "18"),

    # QUANTIDADE (Número de unidades de pavimentos idênticos)
    ("QUANTIDADE", "", "", "Número de unidades de pavimentos")
])

# Atribuir o novo cabeçalho ao dataframe
df_quadro_area_01.columns = columns[:len(df_quadro_area_01.columns)]  # Ajuste dinâmico para garantir compatibilidade

# Extraindo a coluna 'Quantidade' sem o MultiIndex para garantir o alinhamento
quantidade_col = df_quadro_area_01[('QUANTIDADE', '', '', 'Número de unidades de pavimentos')].values
print(quantidade_col)
soma_quantidade = df_quadro_area_01[('QUANTIDADE', '', '', 'Número de unidades de pavimentos')].sum().item()
print(soma_quantidade)
print(type(soma_quantidade))

# Selecionar as colunas numéricas para multiplicação, exceto a coluna 'Quantidade'
numeric_cols = df_quadro_area_01.drop(columns=[('', '', '', 'Pavimento', '1'), ('QUANTIDADE', '', '', 'Número de unidades de pavimentos')])
# Aplicar a multiplicação para cada valor das colunas numéricas pelo valor da coluna `Quantidade`
df_multiplicado = numeric_cols.mul(quantidade_col, axis=0)
# Calcular a soma das colunas multiplicadas
sum_row = df_multiplicado.sum()

# Adicionar a coluna de Pavimento com valor "Total" na linha de somatório
sum_row[('', '', '', 'Pavimento', '1')] = "Total"
# Adicionar a linha de somatório ao dataframe original
df_quadro_area_01.loc['Total'] = sum_row


# Substituir o valor da coluna 'Quantidade' na linha de somatório
df_quadro_area_01.at['Total', ('QUANTIDADE', '', '', 'Número de unidades de pavimentos')] = soma_quantidade
print(df_quadro_area_01)


# SECOND SUMS
real_private_area_col = ('ÁREA DE DIVISÃO NÃO PROPORCIONAL', 'ÁREA PRIVATIVA', 'TOTAIS', 'REAL (2+3)', '5')
real_common_area_col = ('ÁREA DE DIVISÃO NÃO PROPORCIONAL', 'ÁREA DE USO COMUM', 'TOTAIS', 'REAL (7+8)', '10')
real_proportional_common_area_col = ('ÁREA DE DIVISÃO PROPORCIONAL', 'ÁREA DE USO COMUM', 'TOTAIS', 'REAL (12+13)', '15')

equivalent_private_area_col = ('ÁREA DE DIVISÃO NÃO PROPORCIONAL', 'ÁREA PRIVATIVA', 'TOTAIS', 'EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (2+4)', '6')
equivalent_common_area_col = ('ÁREA DE DIVISÃO NÃO PROPORCIONAL', 'ÁREA DE USO COMUM', 'TOTAIS', 'EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (7+9)', '11')
equivalent_proportional_common_area_col = ('ÁREA DE DIVISÃO PROPORCIONAL', 'ÁREA DE USO COMUM', 'TOTAIS', 'EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (12+14)', '16')

# Calculate the sums
real_private_area_sum = df_quadro_area_01.loc['Total', real_private_area_col].sum()
real_common_area_sum = df_quadro_area_01.loc['Total', real_common_area_col].sum()
real_proportional_common_area_sum = df_quadro_area_01.loc['Total', real_proportional_common_area_col].sum()

equivalent_private_area_sum = df_quadro_area_01.loc['Total', equivalent_private_area_col].sum()
equivalent_common_area_sum = df_quadro_area_01.loc['Total', equivalent_common_area_col].sum()
equivalent_proportional_common_area_sum = df_quadro_area_01.loc['Total', equivalent_proportional_common_area_col].sum()

# Create a new "Total 2" row
df_quadro_area_01.loc['Total 2', ('', '', '', 'Pavimento', '1')] = "ÁREA REAL GLOBAL:"
df_quadro_area_01.loc['Total 3', ('', '', '', 'Pavimento', '1')] = "ÁREA EQUIVALENTE GLOBAL:"


# Update the "Total" row with the correct values
#df_quadro_area_01.loc['Total 2', ('', '', '', 'Pavimento', '1')] = "Total 2"
df_quadro_area_01.loc['Total 2', ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO", "", "2")] = real_private_area_sum + real_common_area_sum + real_proportional_common_area_sum
df_quadro_area_01.loc['Total 3', ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO", "", "2")] = equivalent_private_area_sum + equivalent_common_area_sum + equivalent_proportional_common_area_sum















# Formatar os dados usando pandas Styler
def format_styler(df):
    styler = df.style.set_caption("").set_table_styles(
        [
            {'selector': 'thead th:first-child', 'props': 'display:none'},  # Hide the first column header
            {'selector': 'thead th', 'props': [('background-color', 'lightgrey'), ('text-align', 'center'), ('font-weight', 'bold'), ('display', 'table-cell'), ('border', '1px solid black')]},
            {'selector': 'tbody td', 'props': [('border', '1px solid black'), ('text-align', 'right')]},
            {'selector': '.index_name', 'props': 'display:none'},
            {'selector': '.row_heading', 'props': 'display:none'}
        ]
    ).format(na_rep='-', precision=8, decimal=',', thousands='.')

    return styler

# Aplicar a formatação
styled_table = format_styler(df_quadro_area_01)


# HTML para o cabeçalho antes da tabela, usando os dados obtidos
html_cabecalho = f"""
<table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
            INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
            (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
        </td>
    </tr>
    <tr>
        <td colspan="4" style="text-align: center; border: 1px solid black;">
            QUADRO I - Cálculo das Áreas nos Pavimentos e da Área Global - Colunas 1 a 18
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Local do Imóvel: {local_construcao}, Curitiba, Paraná</td>
        <td style="border: 1px solid black;">Folha 2</td>
        <td style="border: 1px solid black;">Total de Folhas 10</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Nome do edifício: {nome_edificio}</td>
    </tr>
    <tr>
        <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
        <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">{incorporador}</td>
        <td style="border: 1px solid black;">Data: 28/10/2024</td>
        <td style="border: 1px solid black;">{responsavel_tecnico} ({registro_crea})</td>
        <td style="border: 1px solid black;"></td>
    </tr>
</table>
"""



# Gerar o conteúdo completo em HTML, unindo o cabeçalho e a tabela formatada
html_output = html_cabecalho + styled_table._repr_html_()


# Salvar o conteúdo completo em um arquivo HTML
with open("/content/drive/My Drive/Colab_cri/output/nbr_01_quadro_area_01.html", "w") as file:
    file.write(html_output)


[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [3]
 [1]
 [3]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]]
40
<class 'int'>
                                    ÁREA DE DIVISÃO NÃO PROPORCIONAL  \
                                                      ÁREA PRIVATIVA   
                                                      COBERTA PADRÃO   
                          Pavimento                                    
                                  1                                2   
0                         SUBSOLO 3                       115.061925   
1                         SUBSOLO 2                       115.061925   
2                         SUBSOLO 1                         6.071000   
3                            TÉRREO                      1360.000000   
4                          MEZANINO                       676.510000   
5                      2º PAVIMENTO                         0.000000   
6               TIPO B1 (3° PAVTO.)

<ipython-input-16-8e0f0e90cc58>:66: PerformanceWarning: indexing past lexsort depth may impact performance.
  quantidade_col = df_quadro_area_01[('QUANTIDADE', '', '', 'Número de unidades de pavimentos')].values
<ipython-input-16-8e0f0e90cc58>:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  soma_quantidade = df_quadro_area_01[('QUANTIDADE', '', '', 'Número de unidades de pavimentos')].sum().item()
<ipython-input-16-8e0f0e90cc58>:73: PerformanceWarning: indexing past lexsort depth may impact performance.
  numeric_cols = df_quadro_area_01.drop(columns=[('', '', '', 'Pavimento', '1'), ('QUANTIDADE', '', '', 'Número de unidades de pavimentos')])
<ipython-input-16-8e0f0e90cc58>:86: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_quadro_area_01.at['Total', ('QUANTIDADE', '', '', 'Número de unidades de pavimentos')] = soma_quantidade


### Gera o quadro NBR 02

In [17]:
# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela quadro_area_02
query = "SELECT * FROM quadro_area_02"
df_quadro_area_02 = pd.read_sql_query(query, conn)

# Remover a coluna 'ROWID' se ela estiver presente
if 'ROWID' in df_quadro_area_02.columns:
    df_quadro_area_02 = df_quadro_area_02.drop(columns=['ROWID'])

# Buscar as informações da tabela informacoes_preliminares
query = """
SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas
incorporador = result['nome_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
local_construcao = result['local_construcao']
nome_edificio = result['nome_edificio']

# Fechar a conexão com o banco de dados
conn.close()




# Criar as colunas hierárquicas de acordo com a estrutura fornecida, incluindo 'subcondominio'
columns = pd.MultiIndex.from_tuples([
    ("", "", "", "UNIDADE", "19"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO", "", "20"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "REAL", "21"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "EQUIVALENTE", "22"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "TOTAIS", "REAL (20+21)", "23"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (20+22)", "24"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO", "", "25"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "REAL", "26"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "EQUIVALENTE", "27"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "REAL (25+26)", "28"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (25+27)", "29"),
    ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "", "AREA TOTAL EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (24+29)", "30"),
    ("", "", "", "Coeficiente de proporcionalidade (30/Soma30)", "31"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO (31*12)", "", "32"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "REAL (31*13)", "33"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "COBERTA PADRÃO DIFERENTE OU DESCOBERTA", "EQUIVALENTE (31*14)", "34"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "REAL (32+33)", "35"),
    ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (32+34)", "36"),
    ("ÁREA DA UNIDADE", "", "", "REAL (23+28+35)", "37"),
    ("ÁREA DA UNIDADE", "", "", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (30+36)", "38"),
    ("QUANTIDADE", "", "", "(Número de unidades idênticas)")
])


# Right after loading the DataFrame from SQL
df_quadro_area_02 = df_quadro_area_02.drop('subcondominio', axis=1)

# Atribuir o novo cabeçalho ao dataframe
df_quadro_area_02.columns = columns[:len(df_quadro_area_02.columns)]  # Ajuste dinâmico para garantir compatibilidade

# Extraindo a coluna 'Quantidade' sem o MultiIndex para garantir o alinhamento
quantidade_col = df_quadro_area_02[("QUANTIDADE", "", "", "(Número de unidades idênticas)")].values
print(quantidade_col)
soma_quantidade = df_quadro_area_02[("QUANTIDADE", "", "", "(Número de unidades idênticas)")].sum().item()
print(soma_quantidade)
print(type(soma_quantidade))

# Selecionar as colunas numéricas para multiplicação, exceto a coluna 'Quantidade'
numeric_cols = df_quadro_area_02.drop(columns=[
    ("", "", "", "UNIDADE", "19"),
    ("QUANTIDADE", "", "", "(Número de unidades idênticas)")
])
print(numeric_cols)
# Aplicar a multiplicação para cada valor das colunas numéricas pelo valor da coluna `Quantidade`
df_multiplicado = numeric_cols.mul(quantidade_col, axis=0)
# Calcular a soma das colunas multiplicadas
sum_row = df_multiplicado.sum()

# Adicionar a coluna de Pavimento com valor "Total" na linha de somatório
sum_row[("", "", "", "UNIDADE", "19")] = "Total"
# Adicionar a linha de somatório ao dataframe original
df_quadro_area_02.loc['Total'] = sum_row


# Substituir o valor da coluna 'Quantidade' na linha de somatório
df_quadro_area_02.at['Total', ("QUANTIDADE", "", "", "(Número de unidades idênticas)")] = soma_quantidade
print(df_quadro_area_02)




# SECOND SUMS
real_private_area_col = ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "TOTAIS", "REAL (20+21)", "23")
real_common_area_col = ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "REAL (25+26)", "28")
real_proportional_common_area_col = ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "REAL (32+33)", "35")

equivalent_private_area_col = ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (20+22)", "24")
equivalent_common_area_col =("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (25+27)", "29")
equivalent_proportional_common_area_col = ("ÁREA DE DIVISÃO PROPORCIONAL", "ÁREA DE USO COMUM", "TOTAIS", "EQUIVALENTE EM ÁREA DE CUSTO PADRÃO (32+34)", "36")

# Calculate the sums
real_private_area_sum = df_quadro_area_02.loc['Total', real_private_area_col].sum()
real_common_area_sum = df_quadro_area_02.loc['Total', real_common_area_col].sum()
real_proportional_common_area_sum = df_quadro_area_02.loc['Total', real_proportional_common_area_col].sum()

equivalent_private_area_sum = df_quadro_area_02.loc['Total', equivalent_private_area_col].sum()
equivalent_common_area_sum = df_quadro_area_02.loc['Total', equivalent_common_area_col].sum()
equivalent_proportional_common_area_sum = df_quadro_area_02.loc['Total', equivalent_proportional_common_area_col].sum()

# Create a new "Total 2" row
df_quadro_area_02.loc['Total 2', ("", "", "", "UNIDADE", "19")] = "ÁREA REAL GLOBAL:"
df_quadro_area_02.loc['Total 3', ("", "", "", "UNIDADE", "19")] = "ÁREA EQUIVALENTE GLOBAL:"


# Update the "Total" row with the correct values
#df_quadro_area_02.loc['Total 2', ('', '', '', 'Pavimento', '1')] = "Total 2"
df_quadro_area_02.loc['Total 2', ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO", "", "20")] = real_private_area_sum + real_common_area_sum + real_proportional_common_area_sum
df_quadro_area_02.loc['Total 3', ("ÁREA DE DIVISÃO NÃO PROPORCIONAL", "ÁREA PRIVATIVA", "COBERTA PADRÃO", "", "20")] = equivalent_private_area_sum + equivalent_common_area_sum + equivalent_proportional_common_area_sum


# Formatar os dados usando pandas Styler
def format_styler(df):
    styler = df.style.set_caption("").set_table_styles(
        [
            {'selector': 'thead th:first-child', 'props': 'display:none'},  # Hide the first column header
            {'selector': 'thead th', 'props': [('background-color', 'lightgrey'), ('text-align', 'center'), ('font-weight', 'bold'), ('display', 'table-cell'), ('border', '1px solid black')]},
            {'selector': 'tbody td', 'props': [('border', '1px solid black'), ('text-align', 'right')]},
            {'selector': '.index_name', 'props': 'display:none'},
            {'selector': '.row_heading', 'props': 'display:none'}
        ]
    ).format(na_rep='-', precision=8, decimal=',', thousands='.')

    return styler

# Aplicar a formatação
styled_table = format_styler(df_quadro_area_02)

# HTML para o cabeçalho antes da tabela, usando os dados obtidos
html_cabecalho = f"""
<table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
            INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
            (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
        </td>
    </tr>
    <tr>
        <td colspan="4" style="text-align: center; border: 1px solid black;">
            QUADRO II  -  CALCULO DAS ÁREAS DAS UNIDADES AUTÔNOMAS - COLUNAS 19 A 38
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Local do Imóvel: {local_construcao}, Curitiba, Paraná</td>
        <td style="border: 1px solid black;">Folha 3</td>
        <td style="border: 1px solid black;">Total de Folhas 10</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Nome do edifício: {nome_edificio}</td>
    </tr>
    <tr>
        <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
        <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">{incorporador}</td>
        <td style="border: 1px solid black;">Data: 28/10/2024</td>
        <td style="border: 1px solid black;">{responsavel_tecnico} ({registro_crea})</td>
        <td style="border: 1px solid black;"></td>
    </tr>
</table>
"""

# Gerar o conteúdo completo em HTML, unindo o cabeçalho e a tabela formatada
html_output = html_cabecalho + styled_table._repr_html_()

# Salvar o conteúdo completo em um arquivo HTML
with open("/content/drive/My Drive/Colab_cri/output/nbr_02_quadro_area_02.html", "w") as file:
    file.write(html_output)

print("Arquivo HTML gerado com sucesso.")


<ipython-input-17-536adcde0040>:67: PerformanceWarning: indexing past lexsort depth may impact performance.
  quantidade_col = df_quadro_area_02[("QUANTIDADE", "", "", "(Número de unidades idênticas)")].values
<ipython-input-17-536adcde0040>:69: PerformanceWarning: indexing past lexsort depth may impact performance.
  soma_quantidade = df_quadro_area_02[("QUANTIDADE", "", "", "(Número de unidades idênticas)")].sum().item()
<ipython-input-17-536adcde0040>:74: PerformanceWarning: indexing past lexsort depth may impact performance.
  numeric_cols = df_quadro_area_02.drop(columns=[
<ipython-input-17-536adcde0040>:91: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_quadro_area_02.at['Total', ("QUANTIDADE", "", "", "(Número de unidades idênticas)")] = soma_quantidade


[[ 11]
 [  4]
 [ 67]
 [ 70]
 [  6]
 [ 19]
 [ 12]
 [  1]
 [  2]
 [  2]
 [  2]
 [  2]
 [  1]
 [  2]
 [  2]
 [  2]
 [  2]
 [  1]
 [  1]
 [  1]
 [  1]
 [  1]
 [  1]
 [  1]
 [  1]
 [  1]
 [ 22]
 [116]
 [124]
 [ 60]
 [ 22]
 [  8]
 [  8]
 [ 60]
 [ 28]
 [ 10]
 [ 28]
 [ 16]
 [ 16]
 [ 28]
 [ 20]
 [ 20]
 [ 28]
 [ 15]
 [ 60]
 [ 30]
 [ 32]
 [ 30]
 [ 15]
 [ 15]
 [ 15]
 [ 14]
 [ 14]
 [ 14]
 [ 14]]
1098
<class 'int'>
   ÁREA DE DIVISÃO NÃO PROPORCIONAL                                         \
                     ÁREA PRIVATIVA                                          
                     COBERTA PADRÃO COBERTA PADRÃO DIFERENTE OU DESCOBERTA   
                                                                      REAL   
                                 20                                     21   
0                          0.000000                              10.800000   
1                          0.000000                              11.250000   
2                          0.000000              

### Gera o quadro NBR 03

In [18]:
import sqlite3
import pandas as pd
import os
from datetime import datetime

def connect_database(db_path):
    """Establish database connection with error handling"""
    try:
        return sqlite3.connect(db_path)
    except sqlite3.Error as e:
        raise Exception(f"Failed to connect to database: {e}")

def execute_query(conn, query):
    """Execute SQL query with error handling"""
    try:
        return pd.read_sql_query(query, conn)
    except (sqlite3.Error, pd.io.sql.DatabaseError) as e:
        raise Exception(f"Query execution failed: {e}")

def ensure_output_directory(path):
    """Ensure output directory exists"""
    os.makedirs(os.path.dirname(path), exist_ok=True)

def format_styler(df):
    """Format dataframe with styling specific to Quadro 03"""
    # Define the patterns that need indentation
    first_indent_patterns = [
        'CLASSIFICAÇÃO GERAL',
        'USO COMERCIAL',
        'Dependências de uso privativo da unidade autônoma',
        '3.1.', '3.2.',
        '4.1.', '4.2.', '4.3.', '4.4.', '4.5.', '4.6.',
        '5.1.',
        '6.1.', '6.2.', '6.3.', '6.4.', '6.5.', '6.6.',
        '9.1.', '9.2.', '9.3.', '9.4.'
    ]

    second_indent_patterns = [
        'Designação',
        'Padrão de acabamento',
        'Número de pavimentos',
        'Área equivalente total do projeto-padrão adotado (m²)',
        'QUARTOS',
        'SALAS',
        'Banheiros ou WC',
        'Quartos de empregados',
        '6.3.1.', '6.3.2.', '6.3.3.', '6.3.4.', '6.3.5.', '6.3.6.',
        '6.3.7.', '6.3.8.', '6.3.9.', '6.3.10.', '6.3.11.',
        '6.5.1.', '6.5.2.', '6.5.3.', '6.5.4.', '6.5.5.'
    ]

    def indent_item(x):
        if pd.notnull(x):
            if any(str(x).startswith(p) for p in second_indent_patterns):
                return '\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0' + str(x)
            elif any(str(x).startswith(p) for p in first_indent_patterns):
                return '\u00A0\u00A0\u00A0\u00A0' + str(x)
        return x

    def format_number(value, decimals=8):
        """Format number with thousand separator and specified decimals"""
        if pd.isna(value):
            return '-'

        # Format with thousand separator and specified decimals
        formatted = f'{value:,.{decimals}f}'
        # Replace dots and commas correctly for Brazilian format
        formatted = formatted.replace(',', '@').replace('.', ',').replace('@', '.')
        return formatted

    def format_percentage(value):
        """Format number as percentage with 2 decimal places"""
        if pd.isna(value):
            return '-'

        # Convert to percentage and format with 2 decimal places
        percentage = value * 100
        formatted = f'{percentage:.2f}'.replace('.', ',')
        return f'{formatted}%'

    def format_date(value):
        """Format number as date MM/YYYY"""
        if pd.isna(value):
            return '-'
        try:
            # Assuming value is in YYYYMM format as integer
            year = int(value) // 100
            month = int(value) % 100
            return f'{month:02d}/{year}'
        except:
            return str(value)

    # Define format for each cell based on its content and position
    def custom_format(val, item=None, column=None):
        # Handle NaN/None values consistently
        if pd.isna(val):
            return '-'

        # Special case for date
        if str(item).strip() == '3.1. Data' and column == 'valor':
            return format_date(val)

        # For the 'outros' column, format as percentage
        if column == 'outros' and isinstance(val, (int, float)):
            if str(item).strip() in ['4.', '5.', '5.1.']:  # Header rows
                return '-'
            return format_percentage(val)

        if isinstance(val, (int, float)):
            # For all values in 'valor' column
            if column == 'valor':
                if str(item).strip() in ['4.', '5.', '5.1.']:  # Header rows
                    return '-'
                return format_number(val, 2)

            # For percentage values (column 3)
            elif val <= 1:
                return format_number(val, 8)

            # For other numeric values
            else:
                return format_number(val, 8)

        # Convert string 'nan' to '-'
        if str(val).lower() == 'nan':
            return '-'

        return str(val)

    # Apply indentation
    df = df.copy()
    df['item'] = df['item'].apply(indent_item)

    # Format all columns consistently
    for col in df.columns:
        df[col] = df.apply(lambda row: custom_format(row[col], row['item'], col), axis=1)

    return df.style.hide(axis='index').hide(axis='columns').set_table_styles([
        # Overall table styling
        {'selector': 'table', 'props': [
            ('width', '100%'),
            ('border-collapse', 'collapse'),
            ('margin', '0'),
            ('table-layout', 'fixed'),
            ('font-family', 'Arial'),
            ('font-size', '11px')
        ]},
        # Column styling for first column (left-aligned)
        {'selector': 'td:nth-child(1)', 'props': [
            ('text-align', 'left'),
            ('border', '1px solid black'),
            ('padding', '2px 4px'),
            ('width', '60%')
        ]},
        # Column styling for other columns (right-aligned)
        {'selector': 'td:nth-child(2), td:nth-child(3)', 'props': [
            ('text-align', 'right'),
            ('border', '1px solid black'),
            ('padding', '2px 4px'),
            ('width', '20%')
        ]}
    ])

def generate_header_html(info_data):
    """Generate HTML header with proper escaping"""
    return f"""
    <table style="width: 100%; border-collapse: collapse; margin: 0; padding: 0;">
        <tr>
            <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black; padding: 2px 4px;">
                INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
                (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
            </td>
        </tr>
        <tr>
            <td colspan="4" style="text-align: center; border: 1px solid black; padding: 2px 4px;">
                QUADRO III - Avaliação do Custo Global e Unitário da Construção
            </td>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 2px 4px;">Local do Imóvel: {info_data['local_construcao']}, {info_data['cidade_uf']}</td>
            <td style="border: 1px solid black; padding: 2px 4px;">Folha 4</td>
            <td colspan="2" style="border: 1px solid black; padding: 2px 4px;">Total de Folhas 10</td>
        </tr>
        <tr>
        <td style="border: 1px solid black;">Nome do edifício: {info_data['nome_edificio']}</td>
        </tr>
        <tr>
            <td colspan="2" style="border: 1px solid black; padding: 2px 4px;">INCORPORADOR</td>
            <td colspan="2" style="border: 1px solid black; padding: 2px 4px;">PROFISSIONAL RESPONSÁVEL</td>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 2px 4px;">{info_data['nome_incorporador']}</td>
            <td style="border: 1px solid black;">Data: 28/10/2024</td>
            <td colspan="2" style="border: 1px solid black; padding: 2px 4px;">{info_data['nome_responsavel_tecnico']} ({info_data['registro_crea']})</td>
        </tr>
    </table>
    """

def main():
    # Configuration
    db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
    output_path = "/content/drive/My Drive/Colab_cri/output/nbr_03_quadro_area_03.html"

    try:
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database file not found: {db_path}")

        conn = connect_database(db_path)

        try:
            # Load main data
            df_quadro_area_03 = execute_query(conn, "SELECT * FROM quadro_area_03")

            # Clean ROWID if present
            if 'ROWID' in df_quadro_area_03.columns:
                df_quadro_area_03 = df_quadro_area_03.drop(columns=['ROWID'])

            # Load header information
            header_query = """
                SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, cidade_uf, nome_edificio
                FROM informacoes_preliminares
                LIMIT 1
            """
            header_info = execute_query(conn, header_query)

            if header_info.empty:
                raise ValueError("No preliminary information found in database")

            info_data = header_info.iloc[0].to_dict()

            # Apply formatting
            styled_table = format_styler(df_quadro_area_03)

            # Generate complete HTML
            complete_html = f"""
            <!DOCTYPE html>
            <html>
            <head>
                <meta charset="utf-8">
                <style>
                    body {{ margin: 0; padding: 0; font-family: Arial, sans-serif; font-size: 11px; }}
                    table {{ border-spacing: 0; }}
                </style>
            </head>
            <body>
                <div style="width: 100%; margin: 0; padding: 0;">
                    {generate_header_html(info_data)}
                    {styled_table._repr_html_()}
                </div>
            </body>
            </html>
            """

            # Ensure output directory exists
            ensure_output_directory(output_path)

            # Save file
            with open(output_path, "w", encoding='utf-8') as file:
                file.write(complete_html)

            print(f"HTML file successfully generated at: {output_path}")

        finally:
            conn.close()

    except Exception as e:
        print(f"Error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

HTML file successfully generated at: /content/drive/My Drive/Colab_cri/output/nbr_03_quadro_area_03.html


### Gera o quadro NBR 04A

In [19]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela quadro_area_02
query = "SELECT * FROM quadro_area_04A"
df_quadro_area_04 = pd.read_sql_query(query, conn)

# Remover a coluna 'ROWID' se ela estiver presente
if 'ROWID' in df_quadro_area_04.columns:
    df_quadro_area_04 = df_quadro_area_04.drop(columns=['ROWID'])

# Buscar as informações da tabela informacoes_preliminares
query = """
SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas
incorporador = result['nome_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
local_construcao = result['local_construcao']
nome_edificio = result['nome_edificio']

# Fechar a conexão com o banco de dados
conn.close()

# Criar as colunas hierárquicas de acordo com a estrutura fornecida, incluindo 'subcondominio'
columns = pd.MultiIndex.from_tuples([
    ("", "DESIGNAÇÃO DA UNIDADE", "(QII-19)", "39"),

    ("CUSTO DA CONSTRUÇÃO DA UNIDADE AUTÔNOMA", "ÁREA EQUIVALENTE EM ÁREA DE CUSTO PADRÃO DAS UNIDADES", "(QII-38)", "40"),
    ("CUSTO DA CONSTRUÇÃO DA UNIDADE AUTÔNOMA", "CUSTO", "(31 x ITEM 13.QIII)", "41"),
    ("CUSTO DA CONSTRUÇÃO DA UNIDADE AUTÔNOMA", "COEFICIENTE DE PROPORCIONALIDADE (PARA RATEIO DO CUSTO DA CONSTRUÇÃO)", "(QII-31)", "42"),

    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "COEFICIENTE DE PROPORCIONALIDADE (DAS UNIDADES QUE SUPORTAM O CUSTO DA CONSTRUÇÃO)", "(42)", "43"),
    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "COEFICIENTE DE RATEIO DE CONSTRUÇÃO TOTAL", "(43/SOMA 43)", "44"),
    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "ÁREA EQUIVALENTE EM ÁREA DE CUSTO PADRÃO TOTAL", "(44 X SOMA 40)", "45"),
    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "CUSTO DE CONSTRUÇÃO TOTAL", "(44 X ITEM 13.QIII)", "46"),
    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "CUSTO DA SUB-ROGAÇÃO SUPORTADA POR CADA UNIDADE", "(46 - 41)", "47"),
    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "ÁREA REAL DAS UNIDADES SUB-ROGADAS", "(QII-37)", "48"),
    ("RERRATEIO DO CUSTO (QUANDO HOUVER UNIDADE(S) DADA(S) EM PAGAMENTO DO TERRENO)", "QUOTA DA ÁREA REAL DADA EM PAGAMENTO DO TERRENO", "(44 X SOMA 48)", "49"),

    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "TOTAL (TOTAL DE UNIDADES IDÊNTICAS)", "", "50"),
    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "SUB-ROGADAS", "", "51"),
    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "DIFERENCA (UNIDADES QUE SUPORTAM O CUSTO DA EDIFICAÇÃO)", "(50-51)", "52")
])


# Right after loading the DataFrame from SQL
df_quadro_area_04 = df_quadro_area_04.drop('subcondominio', axis=1)

# Atribuir o novo cabeçalho ao dataframe
df_quadro_area_04.columns = columns[:len(df_quadro_area_04.columns)]  # Ajuste dinâmico para garantir compatibilidade

# Extraindo a coluna 'Quantidade' sem o MultiIndex para garantir o alinhamento
quantidade_col = df_quadro_area_04[("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "DIFERENCA (UNIDADES QUE SUPORTAM O CUSTO DA EDIFICAÇÃO)", "(50-51)", "52")].values
print(quantidade_col)

soma_quantidade1 = df_quadro_area_04[("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "TOTAL (TOTAL DE UNIDADES IDÊNTICAS)", "", "50")].sum().item()
print(soma_quantidade1)
print(type(soma_quantidade1))


soma_quantidade2 = df_quadro_area_04[("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "DIFERENCA (UNIDADES QUE SUPORTAM O CUSTO DA EDIFICAÇÃO)", "(50-51)", "52")].sum().item()
print(soma_quantidade2)
print(type(soma_quantidade2))




# Selecionar as colunas numéricas para multiplicação, exceto a coluna 'Quantidade'
numeric_cols = df_quadro_area_04.drop(columns=[
    ("", "DESIGNAÇÃO DA UNIDADE", "(QII-19)", "39"),
    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "TOTAL (TOTAL DE UNIDADES IDÊNTICAS)", "", "50"),
    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "DIFERENCA (UNIDADES QUE SUPORTAM O CUSTO DA EDIFICAÇÃO)", "(50-51)", "52")
])
print(numeric_cols)

# Aplicar a multiplicação para cada valor das colunas numéricas pelo valor da coluna `Quantidade`
df_multiplicado = numeric_cols.mul(quantidade_col, axis=0)
# Calcular a soma das colunas multiplicadas
sum_row = df_multiplicado.sum()

# Identificar a coluna de custo
coluna_custo = ("CUSTO DA CONSTRUÇÃO DA UNIDADE AUTÔNOMA", "CUSTO", "(31 x ITEM 13.QIII)", "41")

# Arredondar apenas o valor desta coluna para duas casas decimais
sum_row[coluna_custo] = round(sum_row[coluna_custo], 2)




print(sum_row)


# Primeiro arredonda os valores numéricos para 2 casas decimais
#sum_row = sum_row.apply(lambda x: round(x, 6) if isinstance(x, (int, float)) else x)
# Adicionar a coluna de Pavimento com valor "Total" na linha de somatório
sum_row[("", "DESIGNAÇÃO DA UNIDADE", "(QII-19)", "39")] = "Total"
# Adicionar a linha de somatório ao dataframe original
df_quadro_area_04.loc['Total'] = sum_row

# Substituir o valor da coluna 'Quantidade' na linha de somatório
df_quadro_area_04.at['Total', ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "TOTAL (TOTAL DE UNIDADES IDÊNTICAS)", "", "50")] = soma_quantidade1
df_quadro_area_04.at['Total', ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "DIFERENCA (UNIDADES QUE SUPORTAM O CUSTO DA EDIFICAÇÃO)", "(50-51)", "52")] = soma_quantidade2
print(df_quadro_area_04)




# Formatar os dados usando pandas Styler
def format_styler(df):
    styler = df.style.set_caption("").set_table_styles(
        [
            {'selector': 'thead th:first-child', 'props': 'display:none'},  # Hide the first column header
            {'selector': 'thead th', 'props': [('background-color', 'lightgrey'), ('text-align', 'center'), ('font-weight', 'bold'), ('display', 'table-cell'), ('border', '1px solid black')]},
            {'selector': 'tbody td', 'props': [('border', '1px solid black'), ('text-align', 'right')]},
            {'selector': '.index_name', 'props': 'display:none'},
            {'selector': '.row_heading', 'props': 'display:none'}
        ]
    ).format(na_rep='-', precision=8, decimal=',', thousands='.')\
    .format({("CUSTO DA CONSTRUÇÃO DA UNIDADE AUTÔNOMA", "CUSTO", "(31 x ITEM 13.QIII)", "41"): '{:,.2f}'.format}, decimal=',', thousands='.')

    return styler

# Aplicar a formatação
styled_table = format_styler(df_quadro_area_04)

# HTML para o cabeçalho antes da tabela, usando os dados obtidos
html_cabecalho = f"""
<table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
            INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
            (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
        </td>
    </tr>
    <tr>
        <td colspan="4" style="text-align: center; border: 1px solid black;">
            QUADRO IV A - AVALIAÇÃO DO CUSTO DE CONSTRUÇÃO DE CADA UNIDADE AUTÔNOMA E CÁCULO DO RE-RATEIO DE SUBROGAÇÃO - COLUNAS 39 A 52
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Local do Imóvel: {local_construcao}, Curitiba, Paraná</td>
        <td style="border: 1px solid black;">Folha 5</td>
        <td style="border: 1px solid black;">Total de Folhas 10</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Nome do edifício: {nome_edificio}</td>
    </tr>
    <tr>
        <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
        <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">{incorporador}</td>
        <td style="border: 1px solid black;">Data: 28/10/2024</td>
        <td style="border: 1px solid black;">{responsavel_tecnico} ({registro_crea})</td>
        <td style="border: 1px solid black;"></td>
    </tr>
</table>
"""

# Gerar o conteúdo completo em HTML, unindo o cabeçalho e a tabela formatada
html_output = html_cabecalho + styled_table._repr_html_()

# Salvar o conteúdo completo em um arquivo HTML
with open("/content/drive/My Drive/Colab_cri/output/nbr_04A_quadro_area_04A.html", "w") as file:
    file.write(html_output)

print("Arquivo HTML gerado com sucesso.")


[ 11   4  67  70   6  19  12   1   2   2   2   2   1   2   2   2   2   1
   1   1   1   1   1   1   1   1  22 116 124  60  22   8   8  60  28  10
  28  16  16  28  20  20  28  15  60  30  32  30  15  15  15  14  14  14
  14]
1098
<class 'int'>
1098
<class 'int'>
                 CUSTO DA CONSTRUÇÃO DA UNIDADE AUTÔNOMA                      \
   ÁREA EQUIVALENTE EM ÁREA DE CUSTO PADRÃO DAS UNIDADES               CUSTO   
                                                (QII-38) (31 x ITEM 13.QIII)   
                                                      40                  41   
0                                           13.818412               42583.27   
1                                           14.394179               44357.70   
2                                           15.353790               47314.58   
3                                           15.993528               49285.35   
4                                           27.636816               85165.07   
5                

### Gera o quadro NBR 04B

In [20]:
# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela quadro_area_02
query = "SELECT * FROM quadro_area_04B"
df_quadro_area_04 = pd.read_sql_query(query, conn)

# Remover a coluna 'ROWID' se ela estiver presente
if 'ROWID' in df_quadro_area_04.columns:
    df_quadro_area_04 = df_quadro_area_04.drop(columns=['ROWID'])

# Buscar as informações da tabela informacoes_preliminares para o cabeçalho
query = """
SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas para o cabeçalho
incorporador = result['nome_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
local_construcao = result['local_construcao']
nome_edificio = result['nome_edificio']

# Fechar a conexão com o banco de dados
conn.close()

# Criar as colunas hierárquicas de acordo com a estrutura fornecida, incluindo 'subcondominio'
columns = pd.MultiIndex.from_tuples([
    ("", "", "subcondominio"),
    ("", "DESIGNAÇÃO DA UNIDADE (19)", "A"),

    ("ÁREAS REAIS", "ÁREA PRIVATIVA PRINCIPAL", "B"),
    ("ÁREAS REAIS", "OUTRAS ÁREAS PRIVATIVAS (ACESSÓRIAS)", "C"),
    ("ÁREAS REAIS", "ÁREA PRIVATIVA TOTAL (23)=(B+C)", "D"),
    ("ÁREAS REAIS", "ÁREA DE USO COMUM (28+35)", "E"),
    ("ÁREAS REAIS", "ÁREA REAL TOTAL (37)=(D+E)", "F"),

    ("", "COEFICIENTE DE PROPORCIONALIDADE (31)", "G"),

    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "", ""),

    ("OBSERVAÇÕES", "", ""),
])

# Atribuir o novo cabeçalho ao dataframe
df_quadro_area_04.columns = columns[:len(df_quadro_area_04.columns)]  # Ajuste dinâmico para garantir compatibilidade

# Extraindo a coluna 'Quantidade' sem o MultiIndex para garantir o alinhamento
quantidade_col = df_quadro_area_04[("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "", "")].values
print(quantidade_col)

soma_quantidade1 = df_quadro_area_04[("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "", "")].sum().item()
print(soma_quantidade1)
print(type(soma_quantidade1))




# Selecionar as colunas numéricas para multiplicação, exceto a coluna 'Quantidade'
numeric_cols = df_quadro_area_04.drop(columns=[
    ("", "DESIGNAÇÃO DA UNIDADE (19)", "A"),
    ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "", ""),
    ("", "", "subcondominio")
])
print(numeric_cols)

# Aplicar a multiplicação para cada valor das colunas numéricas pelo valor da coluna `Quantidade`
df_multiplicado = numeric_cols.mul(quantidade_col, axis=0)
# Calcular a soma das colunas multiplicadas
sum_row = df_multiplicado.sum()

# Adicionar a coluna de Pavimento com valor "Total" na linha de somatório
sum_row[("", "DESIGNAÇÃO DA UNIDADE (19)", "A")] = "Total"
# Adicionar a linha de somatório ao dataframe original
df_quadro_area_04.loc['Total'] = sum_row

# Substituir o valor da coluna 'Quantidade' na linha de somatório
df_quadro_area_04.at['Total', ("QUANTIDADES (Nº DE UNIDADES IDÊNTICAS)", "", "")] = soma_quantidade1

print(df_quadro_area_04)




# Formatar os dados usando pandas Styler
def format_styler(df):
    styler = df.style.set_caption("").set_table_styles(
        [
            {'selector': 'thead th:first-child', 'props': 'display:none'},  # Hide the first column header
            {'selector': 'thead th', 'props': [('background-color', 'lightgrey'), ('text-align', 'center'), ('font-weight', 'bold'), ('display', 'table-cell'), ('border', '1px solid black')]},
            {'selector': 'tbody td', 'props': [('border', '1px solid black'), ('text-align', 'right')]},
            {'selector': '.index_name', 'props': 'display:none'},
            {'selector': '.row_heading', 'props': 'display:none'}
        ]
    ).format(na_rep='-', precision=8, decimal=',', thousands='.')

    return styler

# Aplicar a formatação
styled_table = format_styler(df_quadro_area_04)

# HTML para o cabeçalho antes da tabela, usando os dados obtidos
html_cabecalho = f"""
<table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
            INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
            (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
        </td>
    </tr>
    <tr>
        <td colspan="4" style="text-align: center; border: 1px solid black;">
            QUADRO IV B - RESUMO DAS ÁREAS REAIS PARA ATOS DE REGISTRO E ESCRITURAÇÃO - COLUNAS A a G
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Local do Imóvel: {local_construcao}</td>
        <td style="border: 1px solid black;">Folha 6</td>
        <td style="border: 1px solid black;">Total de Folhas 10</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Nome do edifício: {nome_edificio}</td>
    </tr>
    <tr>
        <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
        <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">{incorporador}</td>
        <td style="border: 1px solid black;">Data: 28/10/2024</td>
        <td style="border: 1px solid black;">{responsavel_tecnico} ({registro_crea})</td>
        <td style="border: 1px solid black;"></td>
    </tr>
</table>
"""

# Gerar o conteúdo completo em HTML, unindo o cabeçalho e a tabela formatada
html_output = html_cabecalho + styled_table._repr_html_()

# Salvar o conteúdo completo em um arquivo HTML
with open("/content/drive/My Drive/Colab_cri/output/nbr_04B_quadro_area_04B.html", "w") as file:
    file.write(html_output)

print("Arquivo HTML gerado com sucesso.")


[ 11   4  67  70   6  19  12   1   2   2   2   2   1   2   2   2   2   1
   1   1   1   1   1   1   1   1  22 116 124  60  22   8   8  60  28  10
  28  16  16  28  20  20  28  15  60  30  32  30  15  15  15  14  14  14
  14]
1098
<class 'int'>
                ÁREAS REAIS                                       \
   ÁREA PRIVATIVA PRINCIPAL OUTRAS ÁREAS PRIVATIVAS (ACESSÓRIAS)   
                          B                                    C   
0                     10.80                             0.000000   
1                     11.25                             0.000000   
2                     12.00                             0.000000   
3                     12.50                             0.000000   
4                     21.60                             0.000000   
5                     22.50                             0.000000   
6                     25.00                             0.000000   
7                     10.80                             6.071000   
8       

### Gera o quadro NBR 05

In [21]:
# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela quadro_area_01
query = "SELECT * FROM quadro_area_05"
df_quadro_area_05 = pd.read_sql_query(query, conn)

# Buscar as informações da tabela informacoes_preliminares
query = """
SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao,
       data_aprovacao_projeto
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas
incorporador = result['nome_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
local_construcao = result['local_construcao']
data_aprovacao = result['data_aprovacao_projeto']


# Fechar a conexão com o banco de dados
conn.close()

# Function to format multiline content
def format_multiline_content(df, column_name):
    values = df[column_name].dropna().tolist()
    return '<br>'.join(values) if values else ''

# Preparar os dados do formulário com múltiplas linhas
form_data = {
    'a) Tipo de edificação:': format_multiline_content(df_quadro_area_05, 'edificacao_tipo'),
    'b) Número de Pavimentos:': format_multiline_content(df_quadro_area_05, 'edificacao_pavimento'),
    'c) Número de unidades autônomas por pavimento:': format_multiline_content(df_quadro_area_05, 'edificacao_quantidade_unidades_por_pavimentos'),
    'd) Explicitação da numeração das unidades autônomas:': format_multiline_content(df_quadro_area_05, 'edificacao_indicacao_unidades_por_pavimento'),
    'e) Pavimentos especiais (situação e descrição):': format_multiline_content(df_quadro_area_05, 'edificacao_descricao_pavimentos'),
    'f) Data de aprovação do projeto e repartição competente:':  format_multiline_content(df_quadro_area_05, 'alvara_construcao_data_emissao'),
    'g) Outras indicações (numero de blocos):': format_multiline_content(df_quadro_area_05, 'edificacao_blocos'),
    'g) Outras indicações (total de unidades autônomas):': format_multiline_content(df_quadro_area_05, 'edificacao_quantidade_unidades_por_subcondominio')
}

# HTML para o cabeçalho e conteúdo principal
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <style>
        body {{ font-family: Arial, sans-serif; }}
        table {{ width: 100%; border-collapse: collapse; margin-bottom: 20px; }}
        td, th {{ border: 1px solid black; padding: 8px; }}
        .header {{ text-align: center; font-weight: bold; }}
        .section-title {{ background-color: #f0f0f0; }}
        .label {{ font-weight: bold; width: 30%; vertical-align: top; }}
        .content {{ text-align: left; }}
        .content-cell {{ text-align: left; vertical-align: top; }}
        .multiline {{
            white-space: pre-line;
            font-style: regular;
        }}
    </style>
</head>
<body>
    <table>
        <tr>
            <td colspan="4" class="header">
                INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
                (LEI 4591 - 16/12/64 - ART. 32 E ABNT NBR 12.721/2006)
            </td>
        </tr>
        <tr>
            <td colspan="3">
                INFORMAÇÕES GERAIS<br>
            </td>
            <td style="text-align: right;">
                FOLHA NÚMERO 7<br>
                TOTAL FOLHAS 10
            </td>
        </tr>
        <tr>
            <td colspan="4">LOCAL DO IMÓVEL: {local_construcao}</td>
        </tr>
        <tr>
            <td colspan="2" class="section-title">INCORPORADOR</td>
            <td colspan="2" class="section-title">PROFISSIONAL RESPONSÁVEL PELO CÁLCULO</td>
        </tr>
        <tr>
            <td colspan="2">
                Nome: {incorporador}<br>
                Data: out/2024
            </td>
            <td colspan="2">
                Nome: {responsavel_tecnico}<br>
                Data: out/2024 &nbsp;&nbsp;&nbsp;&nbsp; Registro no CREA: {registro_crea}
            </td>
        </tr>
    </table>

    <table>
        <tr>
            <td class="label">a) Tipo de edificação:</td>
            <td class="content-cell">{form_data['a) Tipo de edificação:']}</td>
        </tr>
        <tr>
            <td class="label">b) Número de Pavimentos:</td>
            <td class="content-cell">{form_data['b) Número de Pavimentos:']}</td>
        </tr>
        <tr>
            <td class="label">c) Número de unidades autônomas por pavimento:</td>
            <td class="content-cell multiline">{form_data['c) Número de unidades autônomas por pavimento:']}</td>
        </tr>
        <tr>
            <td class="label">d) Explicitação da numeração das unidades autônomas:</td>
            <td class="content-cell multiline">{form_data['d) Explicitação da numeração das unidades autônomas:']}</td>
        </tr>
        <tr>
            <td class="label">e) Pavimentos especiais (situação e descrição):</td>
            <td class="content-cell multiline">{form_data['e) Pavimentos especiais (situação e descrição):']}</td>
        </tr>
        <tr>
            <td class="label">f) Data de aprovação do projeto e repartição competente:</td>
            <td class="content-cell">{form_data['f) Data de aprovação do projeto e repartição competente:']}</td>
        </tr>
        <tr>
            <td class="label">g) Outras indicações (numero de blocos):</td>
            <td class="content-cell">{form_data['g) Outras indicações (numero de blocos):']}</td>
        </tr>
        <tr>
            <td class="label">g) Outras indicações (total de unidades autônomas):</td>
            <td class="content-cell">{form_data['g) Outras indicações (total de unidades autônomas):']}</td>
        </tr>
    </table>
</body>
</html>
"""

# Salvar o conteúdo completo em um arquivo HTML
with open("/content/drive/My Drive/Colab_cri/output/nbr_05_quadro_area_05.html", "w", encoding='utf-8') as file:
    file.write(html_template)

### Gera o quadro NBR 06

In [22]:


def connect_database(db_path):
    """Establish database connection with error handling"""
    try:
        return sqlite3.connect(db_path)
    except sqlite3.Error as e:
        raise Exception(f"Failed to connect to database: {e}")

def execute_query(conn, query):
    """Execute SQL query with error handling"""
    try:
        return pd.read_sql_query(query, conn)
    except (sqlite3.Error, pd.io.sql.DatabaseError) as e:
        raise Exception(f"Query execution failed: {e}")

def ensure_output_directory(path):
    """Ensure output directory exists"""
    os.makedirs(os.path.dirname(path), exist_ok=True)

def validate_dataframe(df, required_columns):
    """Validate dataframe has required columns"""
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

def create_multi_index_columns():
    """Create proper multi-index columns"""
    return pd.MultiIndex.from_tuples([
        ("EQUIPAMENTOS", ""),
        ("TIPO (OU MARCA)", ""),
        ("ACABAMENTO", ""),
        ("DETALHES GERAIS", ""),
    ])

def format_styler(df):
    """Format dataframe with styling"""
    return df.style.set_caption("").set_table_styles([
        {'selector': 'thead th:first-child', 'props': 'display:none'},
        {'selector': 'thead th', 'props': [
            ('background-color', 'lightgrey'),
            ('text-align', 'center'),
            ('font-weight', 'bold'),
            ('display', 'table-cell'),
            ('border', '1px solid black')
        ]},
        {'selector': 'tbody td', 'props': [
            ('border', '1px solid black'),
            ('text-align', 'right')
        ]},
        {'selector': '.index_name', 'props': 'display:none'},
        {'selector': '.row_heading', 'props': 'display:none'}
    ]).format(na_rep='-', precision=8)

def generate_header_html(info_data):
    """Generate HTML header with proper escaping"""
    return f"""
    <table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
        <tr>
            <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
                INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
                (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
            </td>
        </tr>
        <tr>
            <td colspan="4" style="text-align: center; border: 1px solid black;">
                QUADRO VI - MEMORIAL DESCRITIVO DOS EQUIPAMENTOS
            </td>
        </tr>
        <tr>
            <td style="border: 1px solid black;">Local do Imóvel: {info_data['local_construcao']}, Curitiba, Paraná</td>
            <td style="border: 1px solid black;">Folha 8</td>
            <td style="border: 1px solid black;">Total de Folhas 10</td>
        </tr>
        <tr>
            <td style="border: 1px solid black;">Nome do edifício: {info_data['nome_edificio']}</td>
        </tr>
        <tr>
            <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
            <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
        </tr>
        <tr>
            <td style="border: 1px solid black;">{info_data['nome_incorporador']}</td>
            <td style="border: 1px solid black;">Data: 28/10/2024</td>
            <td style="border: 1px solid black;">{info_data['nome_responsavel_tecnico']} ({info_data['registro_crea']})</td>
            <td style="border: 1px solid black;"></td>
        </tr>
    </table>
    """

def main():
    # Configuration
    db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
    output_path = "/content/drive/My Drive/Colab_cri/output/nbr_06_quadro_area_06.html"

    try:
        # Ensure database exists
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database file not found: {db_path}")

        # Connect to database
        conn = connect_database(db_path)

        try:
            # Load main data
            df_quadro_area_06 = execute_query(conn, "SELECT * FROM quadro_area_06")

            # Clean ROWID if present
            if 'ROWID' in df_quadro_area_06.columns:
                df_quadro_area_06 = df_quadro_area_06.drop(columns=['ROWID'])

            # Load header information
            header_query = """
                SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
                FROM informacoes_preliminares
                LIMIT 1
            """
            header_info = execute_query(conn, header_query)

            if header_info.empty:
                raise ValueError("No preliminary information found in database")

            info_data = header_info.iloc[0].to_dict()

            # Set up multi-index columns
            columns = create_multi_index_columns()
            df_quadro_area_06.columns = columns[:len(df_quadro_area_06.columns)]

            # Format table
            styled_table = format_styler(df_quadro_area_06)

            # Generate complete HTML
            header_html = generate_header_html(info_data)
            complete_html = header_html + styled_table._repr_html_()

            # Ensure output directory exists
            ensure_output_directory(output_path)

            # Save file
            with open(output_path, "w", encoding='utf-8') as file:
                file.write(complete_html)

            print(f"HTML file successfully generated at: {output_path}")

        finally:
            conn.close()

    except Exception as e:
        print(f"Error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

HTML file successfully generated at: /content/drive/My Drive/Colab_cri/output/nbr_06_quadro_area_06.html


### Gera o quadro NBR 07

In [23]:


def connect_database(db_path):
    """Establish database connection with error handling"""
    try:
        return sqlite3.connect(db_path)
    except sqlite3.Error as e:
        raise Exception(f"Failed to connect to database: {e}")

def execute_query(conn, query):
    """Execute SQL query with error handling"""
    try:
        return pd.read_sql_query(query, conn)
    except (sqlite3.Error, pd.io.sql.DatabaseError) as e:
        raise Exception(f"Query execution failed: {e}")

def ensure_output_directory(path):
    """Ensure output directory exists"""
    os.makedirs(os.path.dirname(path), exist_ok=True)

def validate_dataframe(df, required_columns):
    """Validate dataframe has required columns"""
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

def create_multi_index_columns():
    """Create proper multi-index columns"""
    return [
        "DEPENDÊNCIAS",
        "PISOS",
        "PAREDES",
        "TETOS",
        "OUTROS"
    ]

def format_styler(df):
    """Format dataframe with styling"""
    return df.style.hide(axis='index').set_table_styles([
        {'selector': 'thead th', 'props': [
            ('background-color', 'lightgrey'),
            ('text-align', 'center'),
            ('font-weight', 'bold'),
            ('display', 'table-cell'),
            ('border', '1px solid black')
        ]},
        {'selector': 'tbody td', 'props': [
            ('border', '1px solid black'),
            ('text-align', 'right')
        ]},
    ]).format(na_rep='-')

def generate_header_html(info_data):
    """Generate HTML header with proper escaping"""
    return f"""
    <table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
        <tr>
            <td colspan="6" style="text-align: center; font-weight: bold; border: 1px solid black;">
                INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
                (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
            </td>
        </tr>
        <tr>
            <td colspan="6" style="text-align: center; border: 1px solid black;">
                QUADRO VII - MEMORIAL DESCRITIVO DOS ACABAMENTOS  (Dependências de uso privativo)
            </td>
        </tr>
        <tr>
            <td colspan="3" style="border: 1px solid black;">Local do Imóvel: {info_data['local_construcao']}, Curitiba, Paraná</td>
            <td style="border: 1px solid black;">Folha 9</td>
            <td colspan="2" style="border: 1px solid black;">Total de Folhas 10</td>
        </tr>
        <tr>
        <td style="border: 1px solid black;">Nome do edifício: {info_data['nome_edificio']}</td>
        </tr>
        <tr>
            <td colspan="3" style="border: 1px solid black;">INCORPORADOR</td>
            <td colspan="3" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
        </tr>
        <tr>
            <td colspan="2" style="border: 1px solid black;">{info_data['nome_incorporador']}</td>
            <td style="border: 1px solid black;">Data: 28/10/2024</td>
            <td colspan="3" style="border: 1px solid black;">{info_data['nome_responsavel_tecnico']} ({info_data['registro_crea']})</td>
        </tr>
    </table>
    """

def main():
    # Configuration
    db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
    output_path = "/content/drive/My Drive/Colab_cri/output/nbr_07_quadro_area_07.html"

    try:
        # Ensure database exists
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database file not found: {db_path}")

        # Connect to database
        conn = connect_database(db_path)

        try:
            # Load main data
            df_quadro_area_07 = execute_query(conn, "SELECT * FROM quadro_area_07")

            # Clean ROWID and subcondominio if present
            if 'ROWID' in df_quadro_area_07.columns:
                df_quadro_area_07 = df_quadro_area_07.drop(columns=['ROWID'])
            if 'subcondominio' in df_quadro_area_07.columns:
                df_quadro_area_07 = df_quadro_area_07.drop(columns=['subcondominio'])

            # Load header information
            header_query = """
                SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
                FROM informacoes_preliminares
                LIMIT 1
            """
            header_info = execute_query(conn, header_query)

            if header_info.empty:
                raise ValueError("No preliminary information found in database")

            info_data = header_info.iloc[0].to_dict()

            # Set up multi-index columns
            columns = create_multi_index_columns()
            df_quadro_area_07.columns = columns[:len(df_quadro_area_07.columns)]

            # Format table
            styled_table = format_styler(df_quadro_area_07)

            # Generate complete HTML
            header_html = generate_header_html(info_data)
            complete_html = header_html + styled_table._repr_html_()

            # Ensure output directory exists
            ensure_output_directory(output_path)

            # Save file
            with open(output_path, "w", encoding='utf-8') as file:
                file.write(complete_html)

            print(f"HTML file successfully generated at: {output_path}")

        finally:
            conn.close()

    except Exception as e:
        print(f"Error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

HTML file successfully generated at: /content/drive/My Drive/Colab_cri/output/nbr_07_quadro_area_07.html


### Gera o quadro NBR 08

In [24]:


def connect_database(db_path):
    """Establish database connection with error handling"""
    try:
        return sqlite3.connect(db_path)
    except sqlite3.Error as e:
        raise Exception(f"Failed to connect to database: {e}")

def execute_query(conn, query):
    """Execute SQL query with error handling"""
    try:
        return pd.read_sql_query(query, conn)
    except (sqlite3.Error, pd.io.sql.DatabaseError) as e:
        raise Exception(f"Query execution failed: {e}")

def ensure_output_directory(path):
    """Ensure output directory exists"""
    os.makedirs(os.path.dirname(path), exist_ok=True)

def validate_dataframe(df, required_columns):
    """Validate dataframe has required columns"""
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

def create_multi_index_columns():
    """Create proper multi-index columns"""
    return [
        "DEPENDÊNCIAS",
        "PISOS",
        "PAREDES",
        "TETOS",
        "OUTROS"
    ]

def format_styler(df):
    """Format dataframe with styling"""
    return df.style.hide(axis='index').set_table_styles([
        {'selector': 'thead th', 'props': [
            ('background-color', 'lightgrey'),
            ('text-align', 'center'),
            ('font-weight', 'bold'),
            ('display', 'table-cell'),
            ('border', '1px solid black')
        ]},
        {'selector': 'tbody td', 'props': [
            ('border', '1px solid black'),
            ('text-align', 'right')
        ]},
    ]).format(na_rep='-')

def generate_header_html(info_data):
    """Generate HTML header with proper escaping"""
    return f"""
    <table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
        <tr>
            <td colspan="6" style="text-align: center; font-weight: bold; border: 1px solid black;">
                INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
                (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
            </td>
        </tr>
        <tr>
            <td colspan="6" style="text-align: center; border: 1px solid black;">
                QUADRO VIII - MEMORIAL DESCRITIVO DOS ACABAMENTOS  (Dependências de uso comum)
            </td>
        </tr>
        <tr>
            <td colspan="3" style="border: 1px solid black;">Local do Imóvel: {info_data['local_construcao']}, Curitiba, Paraná</td>
            <td style="border: 1px solid black;">Folha 10</td>
            <td colspan="2" style="border: 1px solid black;">Total de Folhas 10</td>
        </tr>
        <tr>
        <td style="border: 1px solid black;">Nome do edifício: {info_data['nome_edificio']}</td>
        </tr>
        <tr>
            <td colspan="3" style="border: 1px solid black;">INCORPORADOR</td>
            <td colspan="3" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
        </tr>
        <tr>
            <td colspan="2" style="border: 1px solid black;">{info_data['nome_incorporador']}</td>
            <td style="border: 1px solid black;">Data: 28/10/2024</td>
            <td colspan="3" style="border: 1px solid black;">{info_data['nome_responsavel_tecnico']} ({info_data['registro_crea']})</td>
        </tr>
    </table>
    """

def main():
    # Configuration
    db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
    output_path = "/content/drive/My Drive/Colab_cri/output/nbr_08_quadro_area_08.html"

    try:
        # Ensure database exists
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database file not found: {db_path}")

        # Connect to database
        conn = connect_database(db_path)

        try:
            # Load main data
            df_quadro_area_08 = execute_query(conn, "SELECT * FROM quadro_area_08")

            # Clean ROWID and subcondominio if present
            if 'ROWID' in df_quadro_area_08.columns:
                df_quadro_area_08 = df_quadro_area_08.drop(columns=['ROWID'])
            if 'subcondominio' in df_quadro_area_08.columns:
                df_quadro_area_08 = df_quadro_area_08.drop(columns=['subcondominio'])

            # Load header information
            header_query = """
                SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio
                FROM informacoes_preliminares
                LIMIT 1
            """
            header_info = execute_query(conn, header_query)

            if header_info.empty:
                raise ValueError("No preliminary information found in database")

            info_data = header_info.iloc[0].to_dict()

            # Set up multi-index columns
            columns = create_multi_index_columns()
            df_quadro_area_08.columns = columns[:len(df_quadro_area_08.columns)]

            # Format table
            styled_table = format_styler(df_quadro_area_08)

            # Generate complete HTML
            header_html = generate_header_html(info_data)
            complete_html = header_html + styled_table._repr_html_()

            # Ensure output directory exists
            ensure_output_directory(output_path)

            # Save file
            with open(output_path, "w", encoding='utf-8') as file:
                file.write(complete_html)

            print(f"HTML file successfully generated at: {output_path}")

        finally:
            conn.close()

    except Exception as e:
        print(f"Error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

HTML file successfully generated at: /content/drive/My Drive/Colab_cri/output/nbr_08_quadro_area_08.html


### Gera o quadro resumo

In [25]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
db_path = '/content/drive/My Drive/Colab_cri/base_real.db'
conn = sqlite3.connect(db_path)


# Buscar as informações da tabela informacoes_preliminares
query = """
SELECT nome_incorporador, nome_responsavel_tecnico, registro_crea, local_construcao, nome_edificio, cidade_uf
FROM informacoes_preliminares
LIMIT 1
"""
result = pd.read_sql_query(query, conn).iloc[0]

# Extrair os valores das colunas
incorporador = result['nome_incorporador']
responsavel_tecnico = result['nome_responsavel_tecnico']
registro_crea = result['registro_crea']
local_construcao = result['local_construcao']
nome_edificio = result['nome_edificio']
cidade_uf = result['cidade_uf']



# Carregar os dados da tabela quadro_resumo com os nomes corretos das colunas
query = """
SELECT
    subcondominio,
    unidade_tipo as tipo,
    unidade_quantidade as quantidade,
    area_alvara_privativa,
    area_alvara_deposito_vinculado,
    area_alvara_comum,
    area_alvara_total,
    fracao_ideal_solo_subcondominio,
    area_comum_descoberta,
    area_total,
    fracao_ideal_solo_condominio,
    quota_terreno,
    unidade_numero
FROM quadro_resumo
"""


df_quadro_resumo = pd.read_sql_query(query, conn)


# Criar as colunas hierárquicas depois de carregar os dados corretamente
columns = pd.MultiIndex.from_tuples([
    ("", "", "SUBCONDOMÍNIO"),
    ("", "", "TIPO"),
    ("", "", "QUANT."),
    ("ÁREAS DO ALVARÁ", "", "ÁREA PRIVATIVA"),
    ("ÁREAS DO ALVARÁ", "", "DEPÓSITO VINCULADO"),
    ("ÁREAS DO ALVARÁ", "", "ÁREA COMUM"),
    ("ÁREAS DO ALVARÁ", "", "TOTAL ALVARÁ"),
    ("ÁREAS DO ALVARÁ", "", "FRAÇÃO SUBCONDOMÍNIO"),
    ("", "", "ÁREA COMUM DESCOBERTA"),
    ("", "", "ÁREA TOTAL (COB+DESC)"),
    ("", "", "FRAÇÃO IDEAL DE SOLO"),
    ("", "", "QUOTA DE TERRENO"),
    ("", "", "NÚMERO DA UNIDADE")
])


# Reorganizar as colunas do DataFrame para corresponder ao MultiIndex
df_quadro_resumo.columns = columns




# Formatar os dados usando pandas Styler
def format_styler(df):
    styler = df.style.set_caption("").set_table_styles(
        [
            {'selector': 'thead th:first-child', 'props': 'display:none'},  # Hide the first column header
            {'selector': 'thead th', 'props': [('background-color', 'lightgrey'), ('text-align', 'center'), ('font-weight', 'bold'), ('display', 'table-cell'), ('border', '1px solid black')]},
            {'selector': 'tbody td', 'props': [('border', '1px solid black'), ('text-align', 'right')]},
            {'selector': '.index_name', 'props': 'display:none'},
            {'selector': '.row_heading', 'props': 'display:none'}
        ]
    ).format(na_rep='-', precision=8, decimal=',', thousands='.')

    return styler

# Aplicar a formatação
styled_table = format_styler(df_quadro_resumo)




# ...

# Calcular o subtotal das colunas desejadas
subtotal = df_quadro_resumo.sum(numeric_only=True)

# Adicionar a linha de subtotais diretamente ao DataFrame original usando loc
df_quadro_resumo.loc["Subtotal"] = subtotal

# Preencher as colunas não calculadas com "-"
df_quadro_resumo.loc["Subtotal", ("", "", "SUBCONDOMÍNIO")] = "-"
df_quadro_resumo.loc["Subtotal", ("", "", "TIPO")] = "-"
df_quadro_resumo.loc["Subtotal", ("", "", "QUANT.")] = "Total"
df_quadro_resumo.loc["Subtotal", ("ÁREAS DO ALVARÁ", "", "FRAÇÃO SUBCONDOMÍNIO")] = "-"
df_quadro_resumo.loc["Subtotal", ("", "", "NÚMERO DA UNIDADE")] = "-"



# Aplicar a formatação à tabela atualizada
styled_table = format_styler(df_quadro_resumo)

# ...


# HTML para o cabeçalho antes da tabela, usando os dados obtidos
html_cabecalho = f"""
<table style="width: 100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <td colspan="4" style="text-align: center; font-weight: bold; border: 1px solid black;">
            INFORMAÇÕES PARA ARQUIVO NO REGISTRO DE IMÓVEIS<br>
            (Lei 4.591 - 16/12/64 - Art. 32 e ABNT NBR 12721)
        </td>
    </tr>
    <tr>
        <td colspan="4" style="text-align: center; border: 1px solid black;">
            QUADRO RESUMO
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Local do Imóvel: {local_construcao}, {cidade_uf}</td>
        <td style="border: 1px solid black;">FOLHA ÚNICA</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">Nome do edifício: {nome_edificio}</td>
    </tr>
    <tr>
        <td colspan="2" style="border: 1px solid black;">INCORPORADOR</td>
        <td colspan="2" style="border: 1px solid black;">PROFISSIONAL RESPONSÁVEL</td>
    </tr>
    <tr>
        <td style="border: 1px solid black;">{incorporador}</td>
        <td style="border: 1px solid black;">Data: 28/10/2024</td>
        <td style="border: 1px solid black;">{responsavel_tecnico} ({registro_crea})</td>
        <td style="border: 1px solid black;"></td>
    </tr>
</table>
"""

# Gerar o conteúdo completo em HTML, unindo o cabeçalho e a tabela formatada
html_output = html_cabecalho + styled_table._repr_html_()

# Salvar o conteúdo completo em um arquivo HTML
with open("/content/drive/My Drive/Colab_cri/output/nbr_00_quadro_resumo.html", "w") as file:
    file.write(html_output)


conn.close()

print("Arquivo HTML gerado com sucesso.")



<ipython-input-25-b0ef095f1895>:105: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Total' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_quadro_resumo.loc["Subtotal", ("", "", "QUANT.")] = "Total"
<ipython-input-25-b0ef095f1895>:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_quadro_resumo.loc["Subtotal", ("ÁREAS DO ALVARÁ", "", "FRAÇÃO SUBCONDOMÍNIO")] = "-"


Arquivo HTML gerado com sucesso.


## Gera o texto para o registro de incorporação

In [40]:


# AJUTES:
# descrever somente unitario, sem tipo
# usar base igual do memorial

# colocar o Acesso do memorial no final.

import sqlite3
import pandas as pd
from pathlib import Path
import locale


def connect_db(db_path='/content/drive/My Drive/Colab_cri/base_real.db'):
    """Estabelece conexão com o banco de dados."""
    return sqlite3.connect(db_path)

def get_preliminary_info(conn):
    """Obtém informações preliminares da incorporação."""
    query = """
    SELECT
        nome_incorporador,
        cnpj_incorporador,
        nome_edificio,
        quantidade_unidades_autonomas,
        area_lote,
        numero_alvara_projeto
    FROM informacoes_preliminares
    LIMIT 1
    """
    return pd.read_sql_query(query, conn).iloc[0]

def get_total_area(conn):
    """Obtém a área total construída do alvará."""
    query = """
    SELECT
        outras_areas_total_global as area_total
    FROM alvara
    LIMIT 1
    """
    return pd.read_sql_query(query, conn).iloc[0]['area_total']

def get_global_cost(conn):
    """Obtains the global cost of construction."""
    query = """
    SELECT valor
    FROM quadro_area_03
    WHERE ROWID = 67
    """
    result = pd.read_sql_query(query, conn)
    if not result.empty and result.iloc[0]['valor'] is not None:
        valor = result.iloc[0]['valor']
        # Ensure 'valor' is a float
        valor = float(str(valor).replace(' ', ''))
        return valor
    return None



def format_brazilian_currency(value):
    """Formats a number to the Brazilian currency standard (e.g., 1.234,56)."""
    if value is None:
        return None

    # Attempt to convert value directly to float
    try:
        value = float(str(value).replace(' ', ''))
    except ValueError:
        # If direct conversion fails, handle Brazilian format
        try:
            value_clean = str(value).replace('.', '').replace(',', '.').replace(' ', '')
            value = float(value_clean)
        except ValueError:
            return None  # Could not parse value

    # Format the number to two decimal places
    parts = f"{value:.2f}".split('.')

    # Format the integer part with periods
    integer_part = ""
    for i, digit in enumerate(reversed(parts[0])):
        if i != 0 and i % 3 == 0:
            integer_part = '.' + integer_part
        integer_part = digit + integer_part

    # Return the formatted number with a comma as the decimal separator
    return f"{integer_part},{parts[1]}"






def get_subcondominios_summary(conn):
    """Obtém o resumo dos subcondomínios do quadro_resumo."""
    query = """
    WITH subcondominio_counts AS (
        SELECT
            subcondominio,
            COUNT(DISTINCT unidade_numero) as total_unidades
        FROM quadro_resumo
        GROUP BY subcondominio
    )
    SELECT
        subcondominio,
        total_unidades
    FROM subcondominio_counts
    ORDER BY subcondominio
    """
    return pd.read_sql_query(query, conn)



def clean_unit_description(text):
    """
    Limpa a formatação do texto, removendo quebras de linha desnecessárias
    e mantendo apenas a estrutura essencial.
    """
    # Divide o texto em linhas
    lines = text.strip().split('\n')
    cleaned_lines = []
    current_line = []

    for line in lines:
        line = line.strip()
        if not line:  # Linha vazia
            if current_line:
                cleaned_lines.append(' '.join(current_line))
                current_line = []
            cleaned_lines.append('')
        elif line.startswith('APARTAMENTO') or line.startswith('LOJA') or line.startswith('VAGA') or line.startswith('KITINETE') :
            # Se já temos uma linha atual, salvamos ela primeiro
            if current_line:
                cleaned_lines.append(' '.join(current_line))
                current_line = []
            cleaned_lines.append(line)
        else:
            # Remove caracteres especiais de formatação
            line = line.replace('|', '').strip()
            if line:
                current_line.append(line)

    # Adiciona a última linha se existir
    if current_line:
        cleaned_lines.append(' '.join(current_line))

    # Junta as linhas com a formatação adequada
    final_text = []
    for line in cleaned_lines:
        if line.startswith(('APARTAMENTO', 'LOJA', 'VAGA', 'KITINETE')):
            if final_text:  # Adiciona uma linha em branco antes de nova seção
                final_text.append('')
            final_text.append(line)
        elif line:  # Linha não vazia que não é cabeçalho
            final_text.append(line)

    return ' '.join(final_text)

def read_unit_descriptions():
    """Lê as descrições das unidades dos arquivos preexistentes."""
    base_path = Path('/content/drive/My Drive/Colab_cri/output')

    unit_texts = []

    # Lista de arquivos a serem lidos
    files = [
        ('estacionamento.txt', 'VAGAS'),
        ('galeria.txt', 'LOJAS'),
        ('residencial.txt', 'KITINETES')
  ]

    for filename, section_title in files:
        file_path = base_path / filename
        if file_path.exists():
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                if content:  # Só adiciona se houver conteúdo
                    cleaned_content = clean_unit_description(content)
                    unit_texts.append(f"\n{section_title}\n{cleaned_content}")

    return "\n".join(unit_texts)


# Define the path to the 'acesso_edificio.txt' file
acesso_edificio_path = Path('/content/drive/My Drive/Colab_cri/output/acesso_edificio.txt')

# Read the content of the file
try:
    acesso_edificio_content = acesso_edificio_path.read_text(encoding='utf-8').strip()
except FileNotFoundError:
    acesso_edificio_content = 'N/A'
    logger.warning(f"File not found: {acesso_edificio_path}")
except Exception as e:
    acesso_edificio_content = f"Error reading file: {e}"
    logger.error(f"Error reading file {acesso_edificio_path}: {e}")




def generate_incorporation_text(db_path='/content/drive/My Drive/Colab_cri/base_real.db'):
    """Generates the complete incorporation registry text."""
    conn = connect_db(db_path)

    # Retrieve all necessary information
    info = get_preliminary_info(conn)
    area_total = get_total_area(conn)
    subcondominios = get_subcondominios_summary(conn)
    custo_global = get_global_cost(conn)

    # Format the subcondominiums text
    subcondominio_text = "\n".join([
        f"({i+1}) {row.subcondominio}: {row.total_unidades} unidades,"
        for i, row in subcondominios.iterrows()
    ])

    # Read unit descriptions from files
    units_text = read_unit_descriptions()

    # Prepare the global cost text
    if custo_global is not None:
        custo_formatado = format_brazilian_currency(custo_global)
        custo_text = f"CUSTO GLOBAL DA CONSTRUÇÃO - O custo global da construção será de R$ {custo_formatado}"
    else:
        custo_text = "CUSTO GLOBAL DA CONSTRUÇÃO - O custo global da construção será apurado ao final da obra"

    # Assemble the complete text
    incorporation_text = f"""INCORPORAÇÃO) -
À vista do que consta no memorial de incorporação subscrito pela firma {info.nome_incorporador} ({info.cnpj_incorporador}),
memorial esse que fica arquivado digitalmente nesta Serventia sob nº [PROTOCOLO]-I, procedo este registro para consignar que no terreno a que se refere esta matrícula será construído
sob o regime de incorporação, nos termos da Lei nº 4.591/64 e posteriores alterações, um condomínio a ser denominado "{info.nome_edificio}",
aprovado pelo Alvará nº {info.numero_alvara_projeto}, com a área total a ser construída de {area_total}m²,
será composto de unidades autônomas de propriedade exclusiva e partes comuns, divididas em {len(subcondominios)} subcondomínios:
{subcondominio_text}
que serão assim identificadas e caracterizadas:
UNIDADES AUTONOMAS: {units_text} . Acesso: {acesso_edificio_content}PRAZO DE CARÊNCIA - 180 (cento e oitenta) dias; {custo_text}

[[DADOS DO CRI]]
[Dados do CRI serão preenchidos pelo cartório]"""

    conn.close()
    return incorporation_text



def save_incorporation_text(text, output_path='/content/drive/My Drive/Colab_cri/output/registro_incorporacao2.txt'):
    """Salva o texto do registro em um arquivo."""
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)

if __name__ == "__main__":
    try:
        texto = generate_incorporation_text()
        save_incorporation_text(texto)
        print("Arquivo de registro de incorporação gerado com sucesso!")
    except Exception as e:
        print(f"Erro ao gerar o registro: {str(e)}")

Arquivo de registro de incorporação gerado com sucesso!
